In [ ]:
pip install plotly

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import statsmodels 
import requests
from bs4 import BeautifulSoup
from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys
from selenium import webdriver 

import jieba
import jieba.analyse
jieba.initialize()
jieba.set_dictionary('dict.txt')

import yfinance as yf 

import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
import random

import numpy as np
import tensorflow as tf
import pandas as pd
pd.options.mode.chained_assignment = None
import seaborn as sns
from matplotlib.pylab import rcParams
import matplotlib.pyplot as plt
import plotly
import plotly.express as px
import plotly.graph_objects as go
import datetime as dt

from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import pyLDAvis
from __future__ import print_function
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
from sklearn.datasets import fetch_20newsgroups
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.covariance import EllipticEnvelope
from sklearn.ensemble import IsolationForest

%matplotlib inline

sns.set(style='whitegrid', palette='muted')
rcParams['figure.figsize'] = 14, 8
np.random.seed(0)
tf.random.set_seed(0)

import warnings
warnings.filterwarnings('ignore')

Building prefix dict from C:\Users\Jane Wong\OneDrive - HKUST Connect\ESG\dict.txt ...
Loading model from cache C:\Users\JANEWO~1\AppData\Local\Temp\jieba.u4dcdda7161e8cb4bbea6dffafa6acf12.cache
Loading model cost 1.510 seconds.
Prefix dict has been built successfully.


In [ ]:
import matplotlib
print(matplotlib.matplotlib_fname())

C:\Users\Jane Wong\anaconda3\lib\site-packages\matplotlib\mpl-data\matplotlibrc


# Import Preprocessed Headlines

In [ ]:
#Import Pre-processed headliens
headline_df = pd.read_csv('headline_df.csv')
headline_df = headline_df.drop(columns = ['Unnamed: 0'])
headline_df['date'] = pd.to_datetime(headline_df['date'])
headline_df

,date,headline,segmented headline,keywords
0,2017-01-01,房委會綠建環評社區獲鉑金評級,房委會 綠建 環評 社區 獲 鉑金 評級,房委會 綠建 環評 社區 鉑金 評級
1,2017-01-01,發祥街西項目獲鉑金評級,發祥 街西 項目 獲 鉑金 評級,發祥 街西 項目 鉑金 評級
2,2017-01-01,潘基文告別寄語聯國為無聲者吶喊,潘基文 告別 寄語 聯國為 無聲 者 吶喊,告別 寄語 聯國為 無聲 吶喊 潘基文
3,2017-01-01,潘基文告別聯合國,潘基文 告別 聯合國,告別 聯合國 潘基文
4,2017-01-01,兩岸國際潘基文告別寄語聯國為無聲者吶喊,兩岸 國際 潘基文 告別 寄語 聯國為 無聲 者 吶喊,兩岸 國際 告別 寄語 聯國為 無聲 吶喊 潘基文
...,...,...,...,...
321941,2022-09-30,俄羅斯指西方涉蓄意破壞北溪天然氣管道美國稱推測元兇為時尚早,俄羅斯 指 西方 涉 蓄意 破壞 北溪 天然氣 管道 美國 稱 推測 元兇 為 時尚 早,俄羅斯 破壞 天然氣 美國 推測 元兇 時尚 北溪 蓄意 管道 西方
321942,2022-09-30,美國辦太平洋島國峰會被指抗衡中國影響力中方重申不搞經濟脅迫,美 國辦 太平洋 島國 峰會 被 指 抗衡 中國 影響力 中方 重申 不 搞 經濟 脅迫,國辦 島國 峰會 中國 影響力 經濟 脅迫 抗衡 重申 中方 太平洋
321943,2022-09-30,澳門旅議會料國慶訪澳旅客增至三萬恢復內地團後旅客人數將再增,澳門 旅 議會 料 國慶 訪澳 旅客 增至 三萬 恢復 內地 團後 旅客 人數 將再 增,旅客 澳門 議會 國慶 訪澳 三萬 恢復 內地 團後 人數 將再 增至
321944,2022-09-30,謝安琪與母女檔拍親子裝大讚囡囡表現專業,謝安琪 與 母女 檔 拍 親子裝 大 讚 囡 囡 表現 專業,謝安琪 親子裝 表現 專業 母女


# Data Partition

In [ ]:
sample_df = headline_df.groupby("date").sample(frac = 0.01 , random_state=1).reset_index()

# Anomaly1: 2018-01(up)
anomaly1 = headline_df[(headline_df['date']>='2017-12-01') & (headline_df['date']<='2017-12-31')]

# Anomaly2: 2018-02(drop)
anomaly2 =  headline_df[(headline_df['date']>='2018-1-01') & (headline_df['date']<='2018-1-31')]

# Anomaly3: 2018-06(drop)
anomaly3 =  headline_df[(headline_df['date']>='2018-5-01') & (headline_df['date']<='2018-5-31')]

#Anomaly4: 2018-10(drop)
anomaly4 =  headline_df[(headline_df['date']>='2018-9-01') & (headline_df['date']<='2018-9-30')]

#Anomaly5: 2019-01(up)
anomaly5 =  headline_df[(headline_df['date']>='2018-12-01') & (headline_df['date']<='2018-12-31')]

#Anomaly6: 2019-05(drop)
anomaly6 =  headline_df[(headline_df['date']>='2019-04-01') & (headline_df['date']<='2019-04-30')]

#Anomaly7: 2019-08(drop)
anomaly7 =  headline_df[(headline_df['date']>='2019-07-01') & (headline_df['date']<='2019-07-31')]

#Anomaly8: 2019-12(up)
anomaly8 =  headline_df[(headline_df['date']>='2019-11-01') & (headline_df['date']<='2019-11-30')]

#Anomaly9: 2020-03(drop)
anomaly9 =  headline_df[(headline_df['date']>='2020-02-01') & (headline_df['date']<='2020-02-29')]

#Anomaly10: 2020-11(up)
anomaly10 =  headline_df[(headline_df['date']>='2020-10-01') & (headline_df['date']<='2020-10-31')]

#Anomaly11: 2021-07(drop)
anomaly11 =  headline_df[(headline_df['date']>='2021-06-01') & (headline_df['date']<='2021-06-30')]

#Anomaly12: 2022-09(drop)
anomaly12 =  headline_df[(headline_df['date']>='2020-08-01') & (headline_df['date']<='2020-08-31')]

positive_anomaly = pd.concat([anomaly1, anomaly5, anomaly8,  anomaly10])
negative_anomaly = pd.concat([anomaly2 , anomaly3 ,anomaly4 , anomaly6 , anomaly7 , anomaly9 , anomaly11 , anomaly12])

In [ ]:
sample_df

,index,date,headline,segmented headline,keywords
0,22,2017-01-01,中國今發布南沙海洋環境預報,中國 今發布 南沙 海洋環境 預報,中國 今發布 海洋環境 預報 南沙
1,113,2017-01-03,辦理低保豈能成吃拿卡要的藉口,辦理 低保 豈能 成 吃拿卡要 的 藉口,吃拿卡要 辦理 豈能 藉口 低保
2,246,2017-01-04,國立傳統藝術中心颱風影響工程對外開放時程延後,國立 傳統 藝術 中心 颱風 影響 工程 對外開放 時程 延後,國立 傳統 藝術 颱風 影響 對外開放 時程 延後 工程 中心
3,335,2017-01-05,六成高空氣污染日數受內地影響環保署不排除霧霾襲港,六成 高 空氣污染 日數 受 內地 影響 環保署 不 排除 霧 霾 襲港,空氣污染 日數 內地 影響 環保署 襲港 六成 排除
4,360,2017-01-06,中國最安全城市排行榜香港第一拉薩第二,中國 最 安全 城市 排行榜 香港 第一 拉薩 第二,中國 拉薩 排行榜 香港 安全 第二 第一 城市
...,...,...,...,...,...
3211,321298,2022-09-28,李家超指當局銳意發展創科注入新經濟動力,李家 超指 當局 銳意 發展 創科 注入 新 經濟 動力,超指 當局 銳意 發展 創科 經濟 動力 李家 注入
3212,321543,2022-09-29,級颶風撲佛州萬人疏散,級 颶風 撲 佛州 萬人 疏散,颶風 佛州 萬人 疏散
3213,321552,2022-09-29,綠色和平極端酷熱工友極危險拒絕紙老虎保障,綠色 和平 極端 酷熱 工友 極 危險 拒絕 紙老虎 保障,綠色 極端 酷熱 危險 拒絕 紙老虎 工友 和平 保障
3214,321840,2022-09-30,美國公布太平洋夥伴關係宣言所羅門群島簽署,美國 公布 太平洋 夥伴關係 宣言 所羅門 群島 簽署,美國 夥伴關係 所羅門 群島 簽署 宣言 太平洋 公布


In [ ]:
def vectorize(df):

    texts = df['keywords'].astype(str).to_list()
#chinese_stopwords=["、","。","〈","〉","《","》","一","一些","一何","一切","一則","一方面","一旦","一來","一樣","一般","一轉眼","七","萬一","三","上","上下","下","不","不僅","不但","不光","不單","不只","不外乎","不如","不妨","不盡","不盡然","不得","不怕","不惟","不成","不拘","不料","不是","不比","不然","不特","不獨","不管","不至於","不若","不論","不過","不問","與","與其","與其說","與否","與此同時","且","且不說","且說","兩者","個","個別","中","臨","為","為了","為什麽","為何","為止","為此","為著","乃","乃至","乃至於","麽","之","之一","之所以","之類","烏乎","乎","乘","九","也","也好","也罷","了","二","二來","於","於是","於是乎","雲雲","雲爾","五","些","亦","人","人們","人家","什","什麽","什麽樣","今","介於","仍","仍舊","從","從此","從而","他","他人","他們","他們們","以","以上","以為","以便","以免","以及","以故","以期","以來","以至","以至於","以致","們","任","任何","任憑","會","似的","但","但凡","但是","何","何以","何況","何處","何時","余外","作為","你","你們","使","使得","例如","依","依據","依照","便於","俺","俺們","倘","倘使","倘或","倘然","倘若","借","借儻然","假使","假如","假若","做","像","兒","先不先","光是","全體","全部","八","六","兮","共","關於","關於具體地說","其","其一","其中","其二","其他","其余","其它","其次","具體地說","具體說來","兼之","內","再","再其次","再則","再有","再者","再者說","再說","冒","沖","況且","幾","幾時","凡","凡是","憑","憑借","出於","出來","分","分別","則","則甚","別","別人","別處","別是","別的","別管","別說","到","前後","前此","前者","加之","加以","即","即令","即使","即便","即如","即或","即若","卻","去","又","又及","及","及其","及至","反之","反而","反過來","反過來說","受到","另","另一方面","另外","另悉","只","只當","只怕","只是","只有","只消","只要","只限","叫","叮咚","可","可以","可是","可見","各","各個","各位","各種","各自","同","同時","後","後者","向","向使","向著","嚇","嗎","否則","吧","吧噠","含","吱","呀","呃","嘔","唄","嗚","嗚呼","呢","呵","呵呵","呸","呼哧","咋","和","咚","咦","咧","咱","咱們","咳","哇","哈","哈哈","哉","哎","哎呀","哎喲","嘩","喲","哦","哩","哪","哪個","哪些","哪兒","哪天","哪年","哪怕","哪樣","哪邊","哪里","哼","哼唷","唉","唯有","啊","啐","啥","啦","啪達","啷當","喂","喏","喔唷","嘍","嗡","嗡嗡","嗬","嗯","噯","嘎","嘎登","噓","嘛","嘻","嘿","嘿嘿","四","因","因為","因了","因此","因著","因而","固然","在","在下","在於","地","基於","處在","多","多麽","多少","大","大家","她","她們","好","如","如上","如上所述","如下","如何","如其","如同","如是","如果","如此","如若","始而","孰料","孰知","寧","寧可","寧願","寧肯","它","它們","對","對於","對待","對方","對比","將","小","爾","爾後","爾爾","尚且","就","就是","就是了","就是說","就算","就要","盡","盡管","盡管如此","豈但","己","已","已矣","巴","巴巴","年","並","並且","庶乎","庶幾","開外","開始","歸","歸齊","當","當地","當然","當著","彼","彼時","彼此","往","待","很","得","得了","怎","怎麽","怎麽辦","怎麽樣","怎奈","怎樣","總之","總的來看","總的來說","總的說來","總而言之","恰恰相反","您","惟其","慢說","我","我們","或","或則","或是","或曰","或者","截至","所","所以","所在","所幸","所有","才","才能","打","打從","把","抑或","拿","按","按照","換句話說","換言之","據","據此","接著","故","故此","故而","旁人","無","無寧","無論","既","既往","既是","既然","日","時","時候","是","是以","是的","更","曾","替","替代","最","月","有","有些","有關","有及","有時","有的","望","朝","朝著","本","本人","本地","本著","本身","來","來著","來自","來說","極了","果然","果真","某","某個","某些","某某","根據","歟","正值","正如","正巧","正是","此","此地","此處","此外","此時","此次","此間","毋寧","每","每當","比","比及","比如","比方","沒奈何","沿","沿著","漫說","焉","然則","然後","然而","照","照著","猶且","猶自","甚且","甚麽","甚或","甚而","甚至","甚至於","用","用來","由","由於","由是","由此","由此可見","的","的確","的話","直到","相對而言","省得","看","眨眼","著","著呢","矣","矣乎","矣哉","離","秒","竟而","第","等","等到","等等","簡言之","管","類如","緊接著","縱","縱令","縱使","縱然","經","經過","結果","給","繼之","繼後","繼而","綜上所述","罷了","者","而","而且","而況","而後","而外","而已","而是","而言","能","能否","騰","自","自個兒","自從","自各兒","自後","自家","自己","自打","自身","至","至於","至今","至若","致","般的","若","若夫","若是","若果","若非","莫不然","莫如","莫若","雖","雖則","雖然","雖說","被","要","要不","要不是","要不然","要麽","要是","譬喻","譬如","讓","許多","論","設使","設或","設若","誠如","誠然","該","說","說來","請","諸","諸位","諸如","誰","誰人","誰料","誰知","賊死","賴以","趕","起","起見","趁","趁著","越是","距","跟","較","較之","邊","過","還","還是","還有","還要","這","這一來","這個","這麽","這麽些","這麽樣","這麽點兒","這些","這會兒","這兒","這就是說","這時","這樣","這次","這般","這邊","這里","進而","連","連同","逐步","通過","遵循","遵照","那","那個","那麽","那麽些","那麽樣","那些","那會兒","那兒","那時","那樣","那般","那邊","那里","都","鄙人","鑒於","針對","阿","除","除了","除外","除開","除此之外","除非","隨","隨後","隨時","隨著","難道說","零","非","非但","非徒","非特","非獨","靠","順","順著","首先"," ","︿","！","＃","＄","％","＆","（","）","＊","＋","，","０","１","２","３","４","５","６","７","８","９","：","；","＜","＞","？","＠","［","］","｛","｜","｝","～","￥"]

# the vectorizer object will be used to transform text to vector form
    vectorizer = TfidfVectorizer(tokenizer=jieba.lcut, token_pattern='\w+|\$[\d\.]+|\S+', use_idf=False)

# apply transformation
    tf = vectorizer.fit_transform(texts)

# tf_feature_names tells us what word each column in the matric represents
    tf_feature_names = vectorizer.get_feature_names()
    
    return tf, tf_feature_names

sample_df = headline_df.groupby("date").sample(frac = 0.01 , random_state=1).reset_index()
tf_sample, tf_feature_names_sample = vectorize(sample_df)

In [ ]:
from sklearn.model_selection import GridSearchCV

# Options to try with our LDA
search_params = {
  'n_components': [2, 3, 4, 8],
  'learning_decay': [0.7]
}

# Set up LDA 
model = LatentDirichletAllocation(learning_method='online')

# Try all of the options
gridsearch = GridSearchCV(model, param_grid=search_params, n_jobs=-1, verbose=1)
gridsearch.fit(tf_sample)

print("Best Model's Params: ", gridsearch.best_params_)
print("Best Log Likelihood Score: ", gridsearch.best_score_)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
Best Model's Params:  {'learning_decay': 0.7, 'n_components': 2}
Best Log Likelihood Score:  -10668.241981968877


# Anomaly Analysis

## Postive Anomaly

In [ ]:
texts = positive_anomaly['keywords'].astype(str).to_list()
#chinese_stopwords=["、","。","〈","〉","《","》","一","一些","一何","一切","一則","一方面","一旦","一來","一樣","一般","一轉眼","七","萬一","三","上","上下","下","不","不僅","不但","不光","不單","不只","不外乎","不如","不妨","不盡","不盡然","不得","不怕","不惟","不成","不拘","不料","不是","不比","不然","不特","不獨","不管","不至於","不若","不論","不過","不問","與","與其","與其說","與否","與此同時","且","且不說","且說","兩者","個","個別","中","臨","為","為了","為什麽","為何","為止","為此","為著","乃","乃至","乃至於","麽","之","之一","之所以","之類","烏乎","乎","乘","九","也","也好","也罷","了","二","二來","於","於是","於是乎","雲雲","雲爾","五","些","亦","人","人們","人家","什","什麽","什麽樣","今","介於","仍","仍舊","從","從此","從而","他","他人","他們","他們們","以","以上","以為","以便","以免","以及","以故","以期","以來","以至","以至於","以致","們","任","任何","任憑","會","似的","但","但凡","但是","何","何以","何況","何處","何時","余外","作為","你","你們","使","使得","例如","依","依據","依照","便於","俺","俺們","倘","倘使","倘或","倘然","倘若","借","借儻然","假使","假如","假若","做","像","兒","先不先","光是","全體","全部","八","六","兮","共","關於","關於具體地說","其","其一","其中","其二","其他","其余","其它","其次","具體地說","具體說來","兼之","內","再","再其次","再則","再有","再者","再者說","再說","冒","沖","況且","幾","幾時","凡","凡是","憑","憑借","出於","出來","分","分別","則","則甚","別","別人","別處","別是","別的","別管","別說","到","前後","前此","前者","加之","加以","即","即令","即使","即便","即如","即或","即若","卻","去","又","又及","及","及其","及至","反之","反而","反過來","反過來說","受到","另","另一方面","另外","另悉","只","只當","只怕","只是","只有","只消","只要","只限","叫","叮咚","可","可以","可是","可見","各","各個","各位","各種","各自","同","同時","後","後者","向","向使","向著","嚇","嗎","否則","吧","吧噠","含","吱","呀","呃","嘔","唄","嗚","嗚呼","呢","呵","呵呵","呸","呼哧","咋","和","咚","咦","咧","咱","咱們","咳","哇","哈","哈哈","哉","哎","哎呀","哎喲","嘩","喲","哦","哩","哪","哪個","哪些","哪兒","哪天","哪年","哪怕","哪樣","哪邊","哪里","哼","哼唷","唉","唯有","啊","啐","啥","啦","啪達","啷當","喂","喏","喔唷","嘍","嗡","嗡嗡","嗬","嗯","噯","嘎","嘎登","噓","嘛","嘻","嘿","嘿嘿","四","因","因為","因了","因此","因著","因而","固然","在","在下","在於","地","基於","處在","多","多麽","多少","大","大家","她","她們","好","如","如上","如上所述","如下","如何","如其","如同","如是","如果","如此","如若","始而","孰料","孰知","寧","寧可","寧願","寧肯","它","它們","對","對於","對待","對方","對比","將","小","爾","爾後","爾爾","尚且","就","就是","就是了","就是說","就算","就要","盡","盡管","盡管如此","豈但","己","已","已矣","巴","巴巴","年","並","並且","庶乎","庶幾","開外","開始","歸","歸齊","當","當地","當然","當著","彼","彼時","彼此","往","待","很","得","得了","怎","怎麽","怎麽辦","怎麽樣","怎奈","怎樣","總之","總的來看","總的來說","總的說來","總而言之","恰恰相反","您","惟其","慢說","我","我們","或","或則","或是","或曰","或者","截至","所","所以","所在","所幸","所有","才","才能","打","打從","把","抑或","拿","按","按照","換句話說","換言之","據","據此","接著","故","故此","故而","旁人","無","無寧","無論","既","既往","既是","既然","日","時","時候","是","是以","是的","更","曾","替","替代","最","月","有","有些","有關","有及","有時","有的","望","朝","朝著","本","本人","本地","本著","本身","來","來著","來自","來說","極了","果然","果真","某","某個","某些","某某","根據","歟","正值","正如","正巧","正是","此","此地","此處","此外","此時","此次","此間","毋寧","每","每當","比","比及","比如","比方","沒奈何","沿","沿著","漫說","焉","然則","然後","然而","照","照著","猶且","猶自","甚且","甚麽","甚或","甚而","甚至","甚至於","用","用來","由","由於","由是","由此","由此可見","的","的確","的話","直到","相對而言","省得","看","眨眼","著","著呢","矣","矣乎","矣哉","離","秒","竟而","第","等","等到","等等","簡言之","管","類如","緊接著","縱","縱令","縱使","縱然","經","經過","結果","給","繼之","繼後","繼而","綜上所述","罷了","者","而","而且","而況","而後","而外","而已","而是","而言","能","能否","騰","自","自個兒","自從","自各兒","自後","自家","自己","自打","自身","至","至於","至今","至若","致","般的","若","若夫","若是","若果","若非","莫不然","莫如","莫若","雖","雖則","雖然","雖說","被","要","要不","要不是","要不然","要麽","要是","譬喻","譬如","讓","許多","論","設使","設或","設若","誠如","誠然","該","說","說來","請","諸","諸位","諸如","誰","誰人","誰料","誰知","賊死","賴以","趕","起","起見","趁","趁著","越是","距","跟","較","較之","邊","過","還","還是","還有","還要","這","這一來","這個","這麽","這麽些","這麽樣","這麽點兒","這些","這會兒","這兒","這就是說","這時","這樣","這次","這般","這邊","這里","進而","連","連同","逐步","通過","遵循","遵照","那","那個","那麽","那麽些","那麽樣","那些","那會兒","那兒","那時","那樣","那般","那邊","那里","都","鄙人","鑒於","針對","阿","除","除了","除外","除開","除此之外","除非","隨","隨後","隨時","隨著","難道說","零","非","非但","非徒","非特","非獨","靠","順","順著","首先"," ","︿","！","＃","＄","％","＆","（","）","＊","＋","，","０","１","２","３","４","５","６","７","８","９","：","；","＜","＞","？","＠","［","］","｛","｜","｝","～","￥"]

# the vectorizer object will be used to transform text to vector form
vectorizer = TfidfVectorizer(tokenizer=jieba.lcut, token_pattern='\w+|\$[\d\.]+|\S+', stop_words = [" "])

# apply transformation
tf_pos = vectorizer.fit_transform(texts)

# tf_feature_names tells us what word each column in the matric represents
tf_feature_namespos = vectorizer.get_feature_names()

#Distribute topics
lda_tfidf = LatentDirichletAllocation(n_components=2, random_state=0)
lda_tfidf.fit(tf_pos)

#Display result
vis = pyLDAvis.sklearn.prepare(lda_tfidf, tf_pos, vectorizer)
pyLDAvis.enable_notebook()
pyLDAvis.display(vis)

In [ ]:
def plot_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        #ID
        top_features_ind = topic.argsort()[: -n_top_words - 1 : -1]
        #Top Features
        top_features = [feature_names[i] for i in top_features_ind]
        print( top_features )
        #Term Frequency
        weights = topic[top_features_ind]
        print( weights )
plot_top_words(lda_tfidf,tf_feature_namespos,30)

['發展', '中國', '香港', '合作', '經濟', '持續', '習近平', '國際', '企業', '印尼', '全球', '海嘯', '創新', '綠色', '環保', '增至', '世界', '論壇', '投資', '深圳', '金融', '科技', '國家', '項目', '舉行', '推動', '市場', '美國', '社會', '地震']
[101.2411081   96.0707096   59.42785118  57.54349857  51.37339908
  50.76843273  46.13342008  44.28069624  43.60323667  40.75336758
  40.19445033  39.68266857  39.33392443  39.19353101  38.84124136
  38.45761399  36.06319632  35.28402145  34.40511358  33.58012288
  33.46073164  33.06982484  32.07327046  31.32372394  29.07798231
  29.00120612  28.38383962  27.2009543   26.44828704  26.34378019]
['天文台', '颱風', '風球', '天氣', '香港', '信號', '沙德爾', '三號', '浪卡', '熱帶', '年度', '日本', '風暴', '氣旋', '澳門', '強風', '改發', '考慮', '失蹤', '明日', '新聞', '天晴', '生效', '八號', '本港', '維持', '乾燥', '威尼斯', '暴雨', '氣溫']
[150.47104956 105.88991908  84.68617347  71.90992239  71.86424153
  64.58738544  55.39163987  54.99363986  54.42447449  49.14438078
  48.62528323  46.6395833   46.1875637   45.96217368  45.84951221
  44.01028648  43.82392868  43.71010794

## Negative Anomaly

In [ ]:
texts = negative_anomaly['keywords'].astype(str).to_list()
#chinese_stopwords=["、","。","〈","〉","《","》","一","一些","一何","一切","一則","一方面","一旦","一來","一樣","一般","一轉眼","七","萬一","三","上","上下","下","不","不僅","不但","不光","不單","不只","不外乎","不如","不妨","不盡","不盡然","不得","不怕","不惟","不成","不拘","不料","不是","不比","不然","不特","不獨","不管","不至於","不若","不論","不過","不問","與","與其","與其說","與否","與此同時","且","且不說","且說","兩者","個","個別","中","臨","為","為了","為什麽","為何","為止","為此","為著","乃","乃至","乃至於","麽","之","之一","之所以","之類","烏乎","乎","乘","九","也","也好","也罷","了","二","二來","於","於是","於是乎","雲雲","雲爾","五","些","亦","人","人們","人家","什","什麽","什麽樣","今","介於","仍","仍舊","從","從此","從而","他","他人","他們","他們們","以","以上","以為","以便","以免","以及","以故","以期","以來","以至","以至於","以致","們","任","任何","任憑","會","似的","但","但凡","但是","何","何以","何況","何處","何時","余外","作為","你","你們","使","使得","例如","依","依據","依照","便於","俺","俺們","倘","倘使","倘或","倘然","倘若","借","借儻然","假使","假如","假若","做","像","兒","先不先","光是","全體","全部","八","六","兮","共","關於","關於具體地說","其","其一","其中","其二","其他","其余","其它","其次","具體地說","具體說來","兼之","內","再","再其次","再則","再有","再者","再者說","再說","冒","沖","況且","幾","幾時","凡","凡是","憑","憑借","出於","出來","分","分別","則","則甚","別","別人","別處","別是","別的","別管","別說","到","前後","前此","前者","加之","加以","即","即令","即使","即便","即如","即或","即若","卻","去","又","又及","及","及其","及至","反之","反而","反過來","反過來說","受到","另","另一方面","另外","另悉","只","只當","只怕","只是","只有","只消","只要","只限","叫","叮咚","可","可以","可是","可見","各","各個","各位","各種","各自","同","同時","後","後者","向","向使","向著","嚇","嗎","否則","吧","吧噠","含","吱","呀","呃","嘔","唄","嗚","嗚呼","呢","呵","呵呵","呸","呼哧","咋","和","咚","咦","咧","咱","咱們","咳","哇","哈","哈哈","哉","哎","哎呀","哎喲","嘩","喲","哦","哩","哪","哪個","哪些","哪兒","哪天","哪年","哪怕","哪樣","哪邊","哪里","哼","哼唷","唉","唯有","啊","啐","啥","啦","啪達","啷當","喂","喏","喔唷","嘍","嗡","嗡嗡","嗬","嗯","噯","嘎","嘎登","噓","嘛","嘻","嘿","嘿嘿","四","因","因為","因了","因此","因著","因而","固然","在","在下","在於","地","基於","處在","多","多麽","多少","大","大家","她","她們","好","如","如上","如上所述","如下","如何","如其","如同","如是","如果","如此","如若","始而","孰料","孰知","寧","寧可","寧願","寧肯","它","它們","對","對於","對待","對方","對比","將","小","爾","爾後","爾爾","尚且","就","就是","就是了","就是說","就算","就要","盡","盡管","盡管如此","豈但","己","已","已矣","巴","巴巴","年","並","並且","庶乎","庶幾","開外","開始","歸","歸齊","當","當地","當然","當著","彼","彼時","彼此","往","待","很","得","得了","怎","怎麽","怎麽辦","怎麽樣","怎奈","怎樣","總之","總的來看","總的來說","總的說來","總而言之","恰恰相反","您","惟其","慢說","我","我們","或","或則","或是","或曰","或者","截至","所","所以","所在","所幸","所有","才","才能","打","打從","把","抑或","拿","按","按照","換句話說","換言之","據","據此","接著","故","故此","故而","旁人","無","無寧","無論","既","既往","既是","既然","日","時","時候","是","是以","是的","更","曾","替","替代","最","月","有","有些","有關","有及","有時","有的","望","朝","朝著","本","本人","本地","本著","本身","來","來著","來自","來說","極了","果然","果真","某","某個","某些","某某","根據","歟","正值","正如","正巧","正是","此","此地","此處","此外","此時","此次","此間","毋寧","每","每當","比","比及","比如","比方","沒奈何","沿","沿著","漫說","焉","然則","然後","然而","照","照著","猶且","猶自","甚且","甚麽","甚或","甚而","甚至","甚至於","用","用來","由","由於","由是","由此","由此可見","的","的確","的話","直到","相對而言","省得","看","眨眼","著","著呢","矣","矣乎","矣哉","離","秒","竟而","第","等","等到","等等","簡言之","管","類如","緊接著","縱","縱令","縱使","縱然","經","經過","結果","給","繼之","繼後","繼而","綜上所述","罷了","者","而","而且","而況","而後","而外","而已","而是","而言","能","能否","騰","自","自個兒","自從","自各兒","自後","自家","自己","自打","自身","至","至於","至今","至若","致","般的","若","若夫","若是","若果","若非","莫不然","莫如","莫若","雖","雖則","雖然","雖說","被","要","要不","要不是","要不然","要麽","要是","譬喻","譬如","讓","許多","論","設使","設或","設若","誠如","誠然","該","說","說來","請","諸","諸位","諸如","誰","誰人","誰料","誰知","賊死","賴以","趕","起","起見","趁","趁著","越是","距","跟","較","較之","邊","過","還","還是","還有","還要","這","這一來","這個","這麽","這麽些","這麽樣","這麽點兒","這些","這會兒","這兒","這就是說","這時","這樣","這次","這般","這邊","這里","進而","連","連同","逐步","通過","遵循","遵照","那","那個","那麽","那麽些","那麽樣","那些","那會兒","那兒","那時","那樣","那般","那邊","那里","都","鄙人","鑒於","針對","阿","除","除了","除外","除開","除此之外","除非","隨","隨後","隨時","隨著","難道說","零","非","非但","非徒","非特","非獨","靠","順","順著","首先"," ","︿","！","＃","＄","％","＆","（","）","＊","＋","，","０","１","２","３","４","５","６","７","８","９","：","；","＜","＞","？","＠","［","］","｛","｜","｝","～","￥"]

# the vectorizer object will be used to transform text to vector form
vectorizer = TfidfVectorizer(tokenizer=jieba.lcut, token_pattern='\w+|\$[\d\.]+|\S+', stop_words = [" "])

# apply transformation
tf_neg = vectorizer.fit_transform(texts)

# tf_feature_names tells us what word each column in the matric represents
tf_feature_namesneg = vectorizer.get_feature_names()

#Distribute topics
lda_tfidf = LatentDirichletAllocation(n_components=2, random_state=0)
lda_tfidf.fit(tf_neg)

#Display result
vis = pyLDAvis.sklearn.prepare(lda_tfidf, tf_neg, vectorizer)
pyLDAvis.enable_notebook()
pyLDAvis.display(vis)

In [ ]:
plot_top_words(lda_tfidf,tf_feature_namesneg,30)

['山竹', '颱風', '日本', '中國', '香港', '機場', '航班', '取消', '關西', '政府', '影響', '風災', '襲港', '服務', '市民', '暴雨', '居民', '北海道', '吹襲', '強颱風', '經濟', '停課', '美國', '中心', '地震', '交通', '大阪', '恢復', '傾瀉', '風暴']
[506.38146773 391.57701053 300.57730284 227.96251346 224.9981018
 221.75379882 209.6725619  205.21691106 184.88193729 177.37250973
 158.0544783  155.8690724  147.58390469 147.27878976 137.56808612
 128.45550673 125.56659391 121.34726591 120.54969733 118.26279574
 118.25879508 117.38318561 115.80386446 115.72894442 115.37733298
 114.62226182 114.05090662 111.74960918 109.65405764 107.00338141]
['天文台', '天氣', '山竹', '颱風', '驟雨', '警告', '雷暴', '酷熱', '暴雨', '風球', '信號', '香港', '最高', '持續', '生效', '發展', '本港', '今年', '氣溫', '未來', '熱帶', '死亡', '澳門', '下午', '今日', '明日', '狂風', '改發', '多雲', '襲港']
[644.88958065 506.62575646 385.47540092 314.86206047 314.11464705
 313.21622594 307.8966413  305.52522511 287.7213258  285.60528845
 264.38707162 256.8883382  241.94238129 230.64453552 214.91650521
 204.68042101 187.79002707 184.47034908 1

## Anomaly1

In [ ]:
texts = anomaly1['keywords'].astype(str).to_list()
#chinese_stopwords=["、","。","〈","〉","《","》","一","一些","一何","一切","一則","一方面","一旦","一來","一樣","一般","一轉眼","七","萬一","三","上","上下","下","不","不僅","不但","不光","不單","不只","不外乎","不如","不妨","不盡","不盡然","不得","不怕","不惟","不成","不拘","不料","不是","不比","不然","不特","不獨","不管","不至於","不若","不論","不過","不問","與","與其","與其說","與否","與此同時","且","且不說","且說","兩者","個","個別","中","臨","為","為了","為什麽","為何","為止","為此","為著","乃","乃至","乃至於","麽","之","之一","之所以","之類","烏乎","乎","乘","九","也","也好","也罷","了","二","二來","於","於是","於是乎","雲雲","雲爾","五","些","亦","人","人們","人家","什","什麽","什麽樣","今","介於","仍","仍舊","從","從此","從而","他","他人","他們","他們們","以","以上","以為","以便","以免","以及","以故","以期","以來","以至","以至於","以致","們","任","任何","任憑","會","似的","但","但凡","但是","何","何以","何況","何處","何時","余外","作為","你","你們","使","使得","例如","依","依據","依照","便於","俺","俺們","倘","倘使","倘或","倘然","倘若","借","借儻然","假使","假如","假若","做","像","兒","先不先","光是","全體","全部","八","六","兮","共","關於","關於具體地說","其","其一","其中","其二","其他","其余","其它","其次","具體地說","具體說來","兼之","內","再","再其次","再則","再有","再者","再者說","再說","冒","沖","況且","幾","幾時","凡","凡是","憑","憑借","出於","出來","分","分別","則","則甚","別","別人","別處","別是","別的","別管","別說","到","前後","前此","前者","加之","加以","即","即令","即使","即便","即如","即或","即若","卻","去","又","又及","及","及其","及至","反之","反而","反過來","反過來說","受到","另","另一方面","另外","另悉","只","只當","只怕","只是","只有","只消","只要","只限","叫","叮咚","可","可以","可是","可見","各","各個","各位","各種","各自","同","同時","後","後者","向","向使","向著","嚇","嗎","否則","吧","吧噠","含","吱","呀","呃","嘔","唄","嗚","嗚呼","呢","呵","呵呵","呸","呼哧","咋","和","咚","咦","咧","咱","咱們","咳","哇","哈","哈哈","哉","哎","哎呀","哎喲","嘩","喲","哦","哩","哪","哪個","哪些","哪兒","哪天","哪年","哪怕","哪樣","哪邊","哪里","哼","哼唷","唉","唯有","啊","啐","啥","啦","啪達","啷當","喂","喏","喔唷","嘍","嗡","嗡嗡","嗬","嗯","噯","嘎","嘎登","噓","嘛","嘻","嘿","嘿嘿","四","因","因為","因了","因此","因著","因而","固然","在","在下","在於","地","基於","處在","多","多麽","多少","大","大家","她","她們","好","如","如上","如上所述","如下","如何","如其","如同","如是","如果","如此","如若","始而","孰料","孰知","寧","寧可","寧願","寧肯","它","它們","對","對於","對待","對方","對比","將","小","爾","爾後","爾爾","尚且","就","就是","就是了","就是說","就算","就要","盡","盡管","盡管如此","豈但","己","已","已矣","巴","巴巴","年","並","並且","庶乎","庶幾","開外","開始","歸","歸齊","當","當地","當然","當著","彼","彼時","彼此","往","待","很","得","得了","怎","怎麽","怎麽辦","怎麽樣","怎奈","怎樣","總之","總的來看","總的來說","總的說來","總而言之","恰恰相反","您","惟其","慢說","我","我們","或","或則","或是","或曰","或者","截至","所","所以","所在","所幸","所有","才","才能","打","打從","把","抑或","拿","按","按照","換句話說","換言之","據","據此","接著","故","故此","故而","旁人","無","無寧","無論","既","既往","既是","既然","日","時","時候","是","是以","是的","更","曾","替","替代","最","月","有","有些","有關","有及","有時","有的","望","朝","朝著","本","本人","本地","本著","本身","來","來著","來自","來說","極了","果然","果真","某","某個","某些","某某","根據","歟","正值","正如","正巧","正是","此","此地","此處","此外","此時","此次","此間","毋寧","每","每當","比","比及","比如","比方","沒奈何","沿","沿著","漫說","焉","然則","然後","然而","照","照著","猶且","猶自","甚且","甚麽","甚或","甚而","甚至","甚至於","用","用來","由","由於","由是","由此","由此可見","的","的確","的話","直到","相對而言","省得","看","眨眼","著","著呢","矣","矣乎","矣哉","離","秒","竟而","第","等","等到","等等","簡言之","管","類如","緊接著","縱","縱令","縱使","縱然","經","經過","結果","給","繼之","繼後","繼而","綜上所述","罷了","者","而","而且","而況","而後","而外","而已","而是","而言","能","能否","騰","自","自個兒","自從","自各兒","自後","自家","自己","自打","自身","至","至於","至今","至若","致","般的","若","若夫","若是","若果","若非","莫不然","莫如","莫若","雖","雖則","雖然","雖說","被","要","要不","要不是","要不然","要麽","要是","譬喻","譬如","讓","許多","論","設使","設或","設若","誠如","誠然","該","說","說來","請","諸","諸位","諸如","誰","誰人","誰料","誰知","賊死","賴以","趕","起","起見","趁","趁著","越是","距","跟","較","較之","邊","過","還","還是","還有","還要","這","這一來","這個","這麽","這麽些","這麽樣","這麽點兒","這些","這會兒","這兒","這就是說","這時","這樣","這次","這般","這邊","這里","進而","連","連同","逐步","通過","遵循","遵照","那","那個","那麽","那麽些","那麽樣","那些","那會兒","那兒","那時","那樣","那般","那邊","那里","都","鄙人","鑒於","針對","阿","除","除了","除外","除開","除此之外","除非","隨","隨後","隨時","隨著","難道說","零","非","非但","非徒","非特","非獨","靠","順","順著","首先"," ","︿","！","＃","＄","％","＆","（","）","＊","＋","，","０","１","２","３","４","５","６","７","８","９","：","；","＜","＞","？","＠","［","］","｛","｜","｝","～","￥"]

# the vectorizer object will be used to transform text to vector form
vectorizer = TfidfVectorizer(tokenizer=jieba.lcut, token_pattern='\w+|\$[\d\.]+|\S+', stop_words = [" "])

# apply transformation
tf_1 = vectorizer.fit_transform(texts)

# tf_feature_names tells us what word each column in the matric represents
tf_feature_names1 = vectorizer.get_feature_names()

#Distribute topics
lda_tfidf = LatentDirichletAllocation(n_components=2, random_state=0)
lda_tfidf.fit(tf_1)

In [ ]:
#Display result
vis = pyLDAvis.sklearn.prepare(lda_tfidf, tf_1, vectorizer)
pyLDAvis.enable_notebook()
pyLDAvis.display(vis)

Building prefix dict from C:\Users\Jane Wong\OneDrive - HKUST Connect\ESG\dict.txt ...
Loading model from cache C:\Users\JANEWO~1\AppData\Local\Temp\jieba.u4dcdda7161e8cb4bbea6dffafa6acf12.cache
Loading model cost 1.502 seconds.
Prefix dict has been built successfully.


In [ ]:
def plot_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        #ID
        top_features_ind = topic.argsort()[: -n_top_words - 1 : -1]
        #Top Features
        top_features = [feature_names[i] for i in top_features_ind]
        print( top_features )
        #Term Frequency
        weights = topic[top_features_ind]
        print( weights )
        
plot_top_words(lda_tfidf,tf_feature_names1,30)


[ 244   75 6015   33 2278 4808 5467 2040 8104 1282 5026 6230  926 4020
  508 6152 2809 3103 2474 2469 5346 1918 2084 2790  809 5428  893 4174
 3776 1933]
['中國', '一路', '經濟', '一帶', '天氣', '海洋公園', '發展', '報告', '香港', '加價', '澳門', '聯合國', '全球', '明年', '今年', '美國', '工作', '影響', '安排', '安全', '生態', '國家', '增長', '崔世安', '億元', '疏散', '內地', '最高', '政府', '國際']
[14.93233549 14.65068642 14.38716795 14.27596208 12.21832112 11.08610891
 10.7183718  10.44307966  9.75482772  9.69973329  9.0128817   7.8936319
  7.58349759  7.37509395  7.24714627  6.95038934  6.92315806  6.50409473
  6.46599433  6.13037681  5.9135075   5.8652221   5.76699568  5.71426195
  5.69323246  5.60521885  5.59778677  5.58425532  5.53228233  5.49487096]
[6461  244 8104 5467 2321 2287 4528 1663 5026 8022 6039 7946 2004 8032
 4087 3497 2933  221 1733 1476 2080 4020 1933 6215 5126  926 1241 6330
  562 6532]
['菲律賓', '中國', '香港', '發展', '失蹤', '天秤', '死亡', '合作', '澳門', '風災', '綠色', '項目', '城市', '颱風', '智慧', '持續', '年度', '世界', '吹襲', '南部', '增至', '明年', '國際', '聖

## Anomaly2

In [ ]:
texts = anomaly2['keywords'].astype(str).to_list()
#chinese_stopwords=["、","。","〈","〉","《","》","一","一些","一何","一切","一則","一方面","一旦","一來","一樣","一般","一轉眼","七","萬一","三","上","上下","下","不","不僅","不但","不光","不單","不只","不外乎","不如","不妨","不盡","不盡然","不得","不怕","不惟","不成","不拘","不料","不是","不比","不然","不特","不獨","不管","不至於","不若","不論","不過","不問","與","與其","與其說","與否","與此同時","且","且不說","且說","兩者","個","個別","中","臨","為","為了","為什麽","為何","為止","為此","為著","乃","乃至","乃至於","麽","之","之一","之所以","之類","烏乎","乎","乘","九","也","也好","也罷","了","二","二來","於","於是","於是乎","雲雲","雲爾","五","些","亦","人","人們","人家","什","什麽","什麽樣","今","介於","仍","仍舊","從","從此","從而","他","他人","他們","他們們","以","以上","以為","以便","以免","以及","以故","以期","以來","以至","以至於","以致","們","任","任何","任憑","會","似的","但","但凡","但是","何","何以","何況","何處","何時","余外","作為","你","你們","使","使得","例如","依","依據","依照","便於","俺","俺們","倘","倘使","倘或","倘然","倘若","借","借儻然","假使","假如","假若","做","像","兒","先不先","光是","全體","全部","八","六","兮","共","關於","關於具體地說","其","其一","其中","其二","其他","其余","其它","其次","具體地說","具體說來","兼之","內","再","再其次","再則","再有","再者","再者說","再說","冒","沖","況且","幾","幾時","凡","凡是","憑","憑借","出於","出來","分","分別","則","則甚","別","別人","別處","別是","別的","別管","別說","到","前後","前此","前者","加之","加以","即","即令","即使","即便","即如","即或","即若","卻","去","又","又及","及","及其","及至","反之","反而","反過來","反過來說","受到","另","另一方面","另外","另悉","只","只當","只怕","只是","只有","只消","只要","只限","叫","叮咚","可","可以","可是","可見","各","各個","各位","各種","各自","同","同時","後","後者","向","向使","向著","嚇","嗎","否則","吧","吧噠","含","吱","呀","呃","嘔","唄","嗚","嗚呼","呢","呵","呵呵","呸","呼哧","咋","和","咚","咦","咧","咱","咱們","咳","哇","哈","哈哈","哉","哎","哎呀","哎喲","嘩","喲","哦","哩","哪","哪個","哪些","哪兒","哪天","哪年","哪怕","哪樣","哪邊","哪里","哼","哼唷","唉","唯有","啊","啐","啥","啦","啪達","啷當","喂","喏","喔唷","嘍","嗡","嗡嗡","嗬","嗯","噯","嘎","嘎登","噓","嘛","嘻","嘿","嘿嘿","四","因","因為","因了","因此","因著","因而","固然","在","在下","在於","地","基於","處在","多","多麽","多少","大","大家","她","她們","好","如","如上","如上所述","如下","如何","如其","如同","如是","如果","如此","如若","始而","孰料","孰知","寧","寧可","寧願","寧肯","它","它們","對","對於","對待","對方","對比","將","小","爾","爾後","爾爾","尚且","就","就是","就是了","就是說","就算","就要","盡","盡管","盡管如此","豈但","己","已","已矣","巴","巴巴","年","並","並且","庶乎","庶幾","開外","開始","歸","歸齊","當","當地","當然","當著","彼","彼時","彼此","往","待","很","得","得了","怎","怎麽","怎麽辦","怎麽樣","怎奈","怎樣","總之","總的來看","總的來說","總的說來","總而言之","恰恰相反","您","惟其","慢說","我","我們","或","或則","或是","或曰","或者","截至","所","所以","所在","所幸","所有","才","才能","打","打從","把","抑或","拿","按","按照","換句話說","換言之","據","據此","接著","故","故此","故而","旁人","無","無寧","無論","既","既往","既是","既然","日","時","時候","是","是以","是的","更","曾","替","替代","最","月","有","有些","有關","有及","有時","有的","望","朝","朝著","本","本人","本地","本著","本身","來","來著","來自","來說","極了","果然","果真","某","某個","某些","某某","根據","歟","正值","正如","正巧","正是","此","此地","此處","此外","此時","此次","此間","毋寧","每","每當","比","比及","比如","比方","沒奈何","沿","沿著","漫說","焉","然則","然後","然而","照","照著","猶且","猶自","甚且","甚麽","甚或","甚而","甚至","甚至於","用","用來","由","由於","由是","由此","由此可見","的","的確","的話","直到","相對而言","省得","看","眨眼","著","著呢","矣","矣乎","矣哉","離","秒","竟而","第","等","等到","等等","簡言之","管","類如","緊接著","縱","縱令","縱使","縱然","經","經過","結果","給","繼之","繼後","繼而","綜上所述","罷了","者","而","而且","而況","而後","而外","而已","而是","而言","能","能否","騰","自","自個兒","自從","自各兒","自後","自家","自己","自打","自身","至","至於","至今","至若","致","般的","若","若夫","若是","若果","若非","莫不然","莫如","莫若","雖","雖則","雖然","雖說","被","要","要不","要不是","要不然","要麽","要是","譬喻","譬如","讓","許多","論","設使","設或","設若","誠如","誠然","該","說","說來","請","諸","諸位","諸如","誰","誰人","誰料","誰知","賊死","賴以","趕","起","起見","趁","趁著","越是","距","跟","較","較之","邊","過","還","還是","還有","還要","這","這一來","這個","這麽","這麽些","這麽樣","這麽點兒","這些","這會兒","這兒","這就是說","這時","這樣","這次","這般","這邊","這里","進而","連","連同","逐步","通過","遵循","遵照","那","那個","那麽","那麽些","那麽樣","那些","那會兒","那兒","那時","那樣","那般","那邊","那里","都","鄙人","鑒於","針對","阿","除","除了","除外","除開","除此之外","除非","隨","隨後","隨時","隨著","難道說","零","非","非但","非徒","非特","非獨","靠","順","順著","首先"," ","︿","！","＃","＄","％","＆","（","）","＊","＋","，","０","１","２","３","４","５","６","７","８","９","：","；","＜","＞","？","＠","［","］","｛","｜","｝","～","￥"]

# the vectorizer object will be used to transform text to vector form
vectorizer = TfidfVectorizer(tokenizer=jieba.lcut, token_pattern='\w+|\$[\d\.]+|\S+', stop_words = [" "])

# apply transformation
tf_2 = vectorizer.fit_transform(texts)

# tf_feature_names tells us what word each column in the matric represents
tf_feature_names2 = vectorizer.get_feature_names()

#Distribute topics
lda_tfidf = LatentDirichletAllocation(n_components=2, random_state=0)
lda_tfidf.fit(tf_2)

#Display result
vis = pyLDAvis.sklearn.prepare(lda_tfidf, tf_2, vectorizer)
pyLDAvis.enable_notebook()
pyLDAvis.display(vis)

In [ ]:
plot_top_words(lda_tfidf,tf_feature_names2,30)

[ 245 5615  794 3551 6188 2763 1454 3977 1356 8430 1951 6350  933 1400
 6213 2043 1246 3488  558 2124 3832 1292 8340 5816 5129 5643   68 4179
 1640 3042]
['中國', '發展', '傾瀉', '持續', '經濟', '山泥', '南加州', '新聞', '北極', '香港', '國際', '美國', '全球', '十大', '綠色', '城市', '創新', '投資', '企業', '增至', '政策', '加州', '風暴', '社會', '澳門', '白皮書', '一路', '暴雨', '合作', '建設']
[42.49585984 30.54598518 19.1661267  18.76194515 18.4235489  17.6693858
 17.49090193 15.83786441 13.80432049 13.77007552 13.24290763 12.85796231
 12.66589547 12.58630029 12.4619346  12.2420816  11.76736737 11.62151361
 11.61599877 11.56307189 11.45511977 11.30966067 10.93050839 10.88353691
 10.62521803 10.40936445 10.38885073 10.37054678 10.35008198 10.32093277]
[2313 1507 8430 2100 2843 4695 4686 6213 1640 2070 3124 4344 4252 7849
 1978 2311 4804 4240 4176 4714 4179 1268 4074  505 5931 6533 4675  216
 8412 4336]
['天氣', '去年', '香港', '塞納河', '巴黎', '水位', '氣溫', '綠色', '合作', '基金', '影響', '李克強', '會議', '金融', '團結', '天文台', '法國', '最高', '暴漲', '水浸', '暴雨', '劉怡翔', '昂坪', '

## Anomaly3

In [ ]:
texts = anomaly3['keywords'].astype(str).to_list()
#chinese_stopwords=["、","。","〈","〉","《","》","一","一些","一何","一切","一則","一方面","一旦","一來","一樣","一般","一轉眼","七","萬一","三","上","上下","下","不","不僅","不但","不光","不單","不只","不外乎","不如","不妨","不盡","不盡然","不得","不怕","不惟","不成","不拘","不料","不是","不比","不然","不特","不獨","不管","不至於","不若","不論","不過","不問","與","與其","與其說","與否","與此同時","且","且不說","且說","兩者","個","個別","中","臨","為","為了","為什麽","為何","為止","為此","為著","乃","乃至","乃至於","麽","之","之一","之所以","之類","烏乎","乎","乘","九","也","也好","也罷","了","二","二來","於","於是","於是乎","雲雲","雲爾","五","些","亦","人","人們","人家","什","什麽","什麽樣","今","介於","仍","仍舊","從","從此","從而","他","他人","他們","他們們","以","以上","以為","以便","以免","以及","以故","以期","以來","以至","以至於","以致","們","任","任何","任憑","會","似的","但","但凡","但是","何","何以","何況","何處","何時","余外","作為","你","你們","使","使得","例如","依","依據","依照","便於","俺","俺們","倘","倘使","倘或","倘然","倘若","借","借儻然","假使","假如","假若","做","像","兒","先不先","光是","全體","全部","八","六","兮","共","關於","關於具體地說","其","其一","其中","其二","其他","其余","其它","其次","具體地說","具體說來","兼之","內","再","再其次","再則","再有","再者","再者說","再說","冒","沖","況且","幾","幾時","凡","凡是","憑","憑借","出於","出來","分","分別","則","則甚","別","別人","別處","別是","別的","別管","別說","到","前後","前此","前者","加之","加以","即","即令","即使","即便","即如","即或","即若","卻","去","又","又及","及","及其","及至","反之","反而","反過來","反過來說","受到","另","另一方面","另外","另悉","只","只當","只怕","只是","只有","只消","只要","只限","叫","叮咚","可","可以","可是","可見","各","各個","各位","各種","各自","同","同時","後","後者","向","向使","向著","嚇","嗎","否則","吧","吧噠","含","吱","呀","呃","嘔","唄","嗚","嗚呼","呢","呵","呵呵","呸","呼哧","咋","和","咚","咦","咧","咱","咱們","咳","哇","哈","哈哈","哉","哎","哎呀","哎喲","嘩","喲","哦","哩","哪","哪個","哪些","哪兒","哪天","哪年","哪怕","哪樣","哪邊","哪里","哼","哼唷","唉","唯有","啊","啐","啥","啦","啪達","啷當","喂","喏","喔唷","嘍","嗡","嗡嗡","嗬","嗯","噯","嘎","嘎登","噓","嘛","嘻","嘿","嘿嘿","四","因","因為","因了","因此","因著","因而","固然","在","在下","在於","地","基於","處在","多","多麽","多少","大","大家","她","她們","好","如","如上","如上所述","如下","如何","如其","如同","如是","如果","如此","如若","始而","孰料","孰知","寧","寧可","寧願","寧肯","它","它們","對","對於","對待","對方","對比","將","小","爾","爾後","爾爾","尚且","就","就是","就是了","就是說","就算","就要","盡","盡管","盡管如此","豈但","己","已","已矣","巴","巴巴","年","並","並且","庶乎","庶幾","開外","開始","歸","歸齊","當","當地","當然","當著","彼","彼時","彼此","往","待","很","得","得了","怎","怎麽","怎麽辦","怎麽樣","怎奈","怎樣","總之","總的來看","總的來說","總的說來","總而言之","恰恰相反","您","惟其","慢說","我","我們","或","或則","或是","或曰","或者","截至","所","所以","所在","所幸","所有","才","才能","打","打從","把","抑或","拿","按","按照","換句話說","換言之","據","據此","接著","故","故此","故而","旁人","無","無寧","無論","既","既往","既是","既然","日","時","時候","是","是以","是的","更","曾","替","替代","最","月","有","有些","有關","有及","有時","有的","望","朝","朝著","本","本人","本地","本著","本身","來","來著","來自","來說","極了","果然","果真","某","某個","某些","某某","根據","歟","正值","正如","正巧","正是","此","此地","此處","此外","此時","此次","此間","毋寧","每","每當","比","比及","比如","比方","沒奈何","沿","沿著","漫說","焉","然則","然後","然而","照","照著","猶且","猶自","甚且","甚麽","甚或","甚而","甚至","甚至於","用","用來","由","由於","由是","由此","由此可見","的","的確","的話","直到","相對而言","省得","看","眨眼","著","著呢","矣","矣乎","矣哉","離","秒","竟而","第","等","等到","等等","簡言之","管","類如","緊接著","縱","縱令","縱使","縱然","經","經過","結果","給","繼之","繼後","繼而","綜上所述","罷了","者","而","而且","而況","而後","而外","而已","而是","而言","能","能否","騰","自","自個兒","自從","自各兒","自後","自家","自己","自打","自身","至","至於","至今","至若","致","般的","若","若夫","若是","若果","若非","莫不然","莫如","莫若","雖","雖則","雖然","雖說","被","要","要不","要不是","要不然","要麽","要是","譬喻","譬如","讓","許多","論","設使","設或","設若","誠如","誠然","該","說","說來","請","諸","諸位","諸如","誰","誰人","誰料","誰知","賊死","賴以","趕","起","起見","趁","趁著","越是","距","跟","較","較之","邊","過","還","還是","還有","還要","這","這一來","這個","這麽","這麽些","這麽樣","這麽點兒","這些","這會兒","這兒","這就是說","這時","這樣","這次","這般","這邊","這里","進而","連","連同","逐步","通過","遵循","遵照","那","那個","那麽","那麽些","那麽樣","那些","那會兒","那兒","那時","那樣","那般","那邊","那里","都","鄙人","鑒於","針對","阿","除","除了","除外","除開","除此之外","除非","隨","隨後","隨時","隨著","難道說","零","非","非但","非徒","非特","非獨","靠","順","順著","首先"," ","︿","！","＃","＄","％","＆","（","）","＊","＋","，","０","１","２","３","４","５","６","７","８","９","：","；","＜","＞","？","＠","［","］","｛","｜","｝","～","￥"]

# the vectorizer object will be used to transform text to vector form
vectorizer = TfidfVectorizer(tokenizer=jieba.lcut, token_pattern='\w+|\$[\d\.]+|\S+', stop_words = [" "])

# apply transformation
tf_3 = vectorizer.fit_transform(texts)

# tf_feature_names tells us what word each column in the matric represents
tf_feature_names3 = vectorizer.get_feature_names()

#Distribute topics
lda_tfidf = LatentDirichletAllocation(n_components=2, random_state=0)
lda_tfidf.fit(tf_3)

#Display result
vis = pyLDAvis.sklearn.prepare(lda_tfidf, tf_3, vectorizer)
pyLDAvis.enable_notebook()
pyLDAvis.display(vis)

In [ ]:
plot_top_words(lda_tfidf,tf_feature_names3,30)

[ 5506  7477   805  6066  2012 11156  6694  8658  8381  6028  7820  6294
   559 10420  5217  4081  4412 10828  5507  3112 10824 10952   292 11027
  3612  3041  2633   325  5316  6264]
['暴雨', '發展', '佛誕', '死亡', '印度', '香港', '溫度', '至少', '美國', '正午', '科技', '沙塵暴', '亞洲', '金融', '新界', '影響', '成立', '雷雨', '暴雨成災', '失蹤', '雷暴', '項目', '世界', '風暴', '局部地區', '天氣', '地盤', '中國', '日間', '汶川']
[24.28740601 21.14360177 18.88467386 18.35668084 18.22759079 14.5546624
 14.06443187 12.86192157 12.64554883 12.1850168  11.30309085 10.82907425
 10.6882978  10.15619752 10.14174138 10.03907714  9.8357761   9.52841732
  9.32172913  9.27595875  8.93463286  8.91379079  8.87344842  8.80850052
  8.79972721  8.78237002  8.753038    8.59103156  8.44763888  8.38482953]
[10329  3041  3032  9422 11270  5578  8100  4656   668 11156 11221   325
  7309  6149 10048  3035   363 10467  7720  5570 11251   210  5577  3771
 10824  5717  3540  7477  5346   215]
['酷熱', '天氣', '天文台', '警告', '高溫', '最高', '紀錄', '持續', '今年', '香港', '驟雨', '中國', '生效', '

## Anomaly4

In [ ]:
texts = anomaly4['keywords'].astype(str).to_list()
#chinese_stopwords=["、","。","〈","〉","《","》","一","一些","一何","一切","一則","一方面","一旦","一來","一樣","一般","一轉眼","七","萬一","三","上","上下","下","不","不僅","不但","不光","不單","不只","不外乎","不如","不妨","不盡","不盡然","不得","不怕","不惟","不成","不拘","不料","不是","不比","不然","不特","不獨","不管","不至於","不若","不論","不過","不問","與","與其","與其說","與否","與此同時","且","且不說","且說","兩者","個","個別","中","臨","為","為了","為什麽","為何","為止","為此","為著","乃","乃至","乃至於","麽","之","之一","之所以","之類","烏乎","乎","乘","九","也","也好","也罷","了","二","二來","於","於是","於是乎","雲雲","雲爾","五","些","亦","人","人們","人家","什","什麽","什麽樣","今","介於","仍","仍舊","從","從此","從而","他","他人","他們","他們們","以","以上","以為","以便","以免","以及","以故","以期","以來","以至","以至於","以致","們","任","任何","任憑","會","似的","但","但凡","但是","何","何以","何況","何處","何時","余外","作為","你","你們","使","使得","例如","依","依據","依照","便於","俺","俺們","倘","倘使","倘或","倘然","倘若","借","借儻然","假使","假如","假若","做","像","兒","先不先","光是","全體","全部","八","六","兮","共","關於","關於具體地說","其","其一","其中","其二","其他","其余","其它","其次","具體地說","具體說來","兼之","內","再","再其次","再則","再有","再者","再者說","再說","冒","沖","況且","幾","幾時","凡","凡是","憑","憑借","出於","出來","分","分別","則","則甚","別","別人","別處","別是","別的","別管","別說","到","前後","前此","前者","加之","加以","即","即令","即使","即便","即如","即或","即若","卻","去","又","又及","及","及其","及至","反之","反而","反過來","反過來說","受到","另","另一方面","另外","另悉","只","只當","只怕","只是","只有","只消","只要","只限","叫","叮咚","可","可以","可是","可見","各","各個","各位","各種","各自","同","同時","後","後者","向","向使","向著","嚇","嗎","否則","吧","吧噠","含","吱","呀","呃","嘔","唄","嗚","嗚呼","呢","呵","呵呵","呸","呼哧","咋","和","咚","咦","咧","咱","咱們","咳","哇","哈","哈哈","哉","哎","哎呀","哎喲","嘩","喲","哦","哩","哪","哪個","哪些","哪兒","哪天","哪年","哪怕","哪樣","哪邊","哪里","哼","哼唷","唉","唯有","啊","啐","啥","啦","啪達","啷當","喂","喏","喔唷","嘍","嗡","嗡嗡","嗬","嗯","噯","嘎","嘎登","噓","嘛","嘻","嘿","嘿嘿","四","因","因為","因了","因此","因著","因而","固然","在","在下","在於","地","基於","處在","多","多麽","多少","大","大家","她","她們","好","如","如上","如上所述","如下","如何","如其","如同","如是","如果","如此","如若","始而","孰料","孰知","寧","寧可","寧願","寧肯","它","它們","對","對於","對待","對方","對比","將","小","爾","爾後","爾爾","尚且","就","就是","就是了","就是說","就算","就要","盡","盡管","盡管如此","豈但","己","已","已矣","巴","巴巴","年","並","並且","庶乎","庶幾","開外","開始","歸","歸齊","當","當地","當然","當著","彼","彼時","彼此","往","待","很","得","得了","怎","怎麽","怎麽辦","怎麽樣","怎奈","怎樣","總之","總的來看","總的來說","總的說來","總而言之","恰恰相反","您","惟其","慢說","我","我們","或","或則","或是","或曰","或者","截至","所","所以","所在","所幸","所有","才","才能","打","打從","把","抑或","拿","按","按照","換句話說","換言之","據","據此","接著","故","故此","故而","旁人","無","無寧","無論","既","既往","既是","既然","日","時","時候","是","是以","是的","更","曾","替","替代","最","月","有","有些","有關","有及","有時","有的","望","朝","朝著","本","本人","本地","本著","本身","來","來著","來自","來說","極了","果然","果真","某","某個","某些","某某","根據","歟","正值","正如","正巧","正是","此","此地","此處","此外","此時","此次","此間","毋寧","每","每當","比","比及","比如","比方","沒奈何","沿","沿著","漫說","焉","然則","然後","然而","照","照著","猶且","猶自","甚且","甚麽","甚或","甚而","甚至","甚至於","用","用來","由","由於","由是","由此","由此可見","的","的確","的話","直到","相對而言","省得","看","眨眼","著","著呢","矣","矣乎","矣哉","離","秒","竟而","第","等","等到","等等","簡言之","管","類如","緊接著","縱","縱令","縱使","縱然","經","經過","結果","給","繼之","繼後","繼而","綜上所述","罷了","者","而","而且","而況","而後","而外","而已","而是","而言","能","能否","騰","自","自個兒","自從","自各兒","自後","自家","自己","自打","自身","至","至於","至今","至若","致","般的","若","若夫","若是","若果","若非","莫不然","莫如","莫若","雖","雖則","雖然","雖說","被","要","要不","要不是","要不然","要麽","要是","譬喻","譬如","讓","許多","論","設使","設或","設若","誠如","誠然","該","說","說來","請","諸","諸位","諸如","誰","誰人","誰料","誰知","賊死","賴以","趕","起","起見","趁","趁著","越是","距","跟","較","較之","邊","過","還","還是","還有","還要","這","這一來","這個","這麽","這麽些","這麽樣","這麽點兒","這些","這會兒","這兒","這就是說","這時","這樣","這次","這般","這邊","這里","進而","連","連同","逐步","通過","遵循","遵照","那","那個","那麽","那麽些","那麽樣","那些","那會兒","那兒","那時","那樣","那般","那邊","那里","都","鄙人","鑒於","針對","阿","除","除了","除外","除開","除此之外","除非","隨","隨後","隨時","隨著","難道說","零","非","非但","非徒","非特","非獨","靠","順","順著","首先"," ","︿","！","＃","＄","％","＆","（","）","＊","＋","，","０","１","２","３","４","５","６","７","８","９","：","；","＜","＞","？","＠","［","］","｛","｜","｝","～","￥"]

# the vectorizer object will be used to transform text to vector form
vectorizer = TfidfVectorizer(tokenizer=jieba.lcut, token_pattern='\w+|\$[\d\.]+|\S+', stop_words = [" "])

# apply transformation
tf_4 = vectorizer.fit_transform(texts)

# tf_feature_names tells us what word each column in the matric represents
tf_feature_names4 = vectorizer.get_feature_names()

#Distribute topics
lda_tfidf = LatentDirichletAllocation(n_components=2, random_state=0)
lda_tfidf.fit(tf_4)

#Display result
vis = pyLDAvis.sklearn.prepare(lda_tfidf, tf_4, vectorizer)
pyLDAvis.enable_notebook()
pyLDAvis.display(vis)

In [ ]:
plot_top_words(lda_tfidf,tf_feature_names4,30)

[13045 14694 25809 21242  4864 26922 27189  7279  4286   782  9831 10299
  6419  6201 26962 25816 16482 15470 12959  6753  2470  6216  9100 26978
  5476 13193  9760  8848 18249 26874]
['日本', '機場', '關西', '航班', '取消', '颱風', '香港', '大阪', '北海道', '中國', '影響', '恢復', '地震', '國泰', '飛燕', '關閉', '滯留', '沖繩', '旅客', '增至', '停課', '國際', '巴士', '飛燕襲', '吹襲', '明日', '強颱風', '山竹', '發展', '風災']
[222.8873012  180.85941396 165.2196237  164.43213342 155.90591351
 130.96400274 125.75119972 103.35845911  90.8505733   75.8147143
  74.37972297  72.89083047  71.37575718  69.37214146  67.42851832
  65.39225781  57.44292264  54.82319061  53.42587854  53.11996002
  52.79890227  52.62454285  52.08152787  50.80162706  50.09151207
  49.97479682  46.81709525  45.8977747   44.05611777  42.92340789]
[ 8848 26922  7340 22200  9760 26883 16705 27189  2216 26929  7348 12384
  9146  5542 15298 18222  8725 26874 20662 12316 14003 24665 16142  5476
  2444  9831 26862 14087 21516  7191]
['山竹', '颱風', '天文台', '襲港', '強颱風', '風球', '澳門', '香港', '

## Anomaly5

In [ ]:
texts = anomaly5['keywords'].astype(str).to_list()
#chinese_stopwords=["、","。","〈","〉","《","》","一","一些","一何","一切","一則","一方面","一旦","一來","一樣","一般","一轉眼","七","萬一","三","上","上下","下","不","不僅","不但","不光","不單","不只","不外乎","不如","不妨","不盡","不盡然","不得","不怕","不惟","不成","不拘","不料","不是","不比","不然","不特","不獨","不管","不至於","不若","不論","不過","不問","與","與其","與其說","與否","與此同時","且","且不說","且說","兩者","個","個別","中","臨","為","為了","為什麽","為何","為止","為此","為著","乃","乃至","乃至於","麽","之","之一","之所以","之類","烏乎","乎","乘","九","也","也好","也罷","了","二","二來","於","於是","於是乎","雲雲","雲爾","五","些","亦","人","人們","人家","什","什麽","什麽樣","今","介於","仍","仍舊","從","從此","從而","他","他人","他們","他們們","以","以上","以為","以便","以免","以及","以故","以期","以來","以至","以至於","以致","們","任","任何","任憑","會","似的","但","但凡","但是","何","何以","何況","何處","何時","余外","作為","你","你們","使","使得","例如","依","依據","依照","便於","俺","俺們","倘","倘使","倘或","倘然","倘若","借","借儻然","假使","假如","假若","做","像","兒","先不先","光是","全體","全部","八","六","兮","共","關於","關於具體地說","其","其一","其中","其二","其他","其余","其它","其次","具體地說","具體說來","兼之","內","再","再其次","再則","再有","再者","再者說","再說","冒","沖","況且","幾","幾時","凡","凡是","憑","憑借","出於","出來","分","分別","則","則甚","別","別人","別處","別是","別的","別管","別說","到","前後","前此","前者","加之","加以","即","即令","即使","即便","即如","即或","即若","卻","去","又","又及","及","及其","及至","反之","反而","反過來","反過來說","受到","另","另一方面","另外","另悉","只","只當","只怕","只是","只有","只消","只要","只限","叫","叮咚","可","可以","可是","可見","各","各個","各位","各種","各自","同","同時","後","後者","向","向使","向著","嚇","嗎","否則","吧","吧噠","含","吱","呀","呃","嘔","唄","嗚","嗚呼","呢","呵","呵呵","呸","呼哧","咋","和","咚","咦","咧","咱","咱們","咳","哇","哈","哈哈","哉","哎","哎呀","哎喲","嘩","喲","哦","哩","哪","哪個","哪些","哪兒","哪天","哪年","哪怕","哪樣","哪邊","哪里","哼","哼唷","唉","唯有","啊","啐","啥","啦","啪達","啷當","喂","喏","喔唷","嘍","嗡","嗡嗡","嗬","嗯","噯","嘎","嘎登","噓","嘛","嘻","嘿","嘿嘿","四","因","因為","因了","因此","因著","因而","固然","在","在下","在於","地","基於","處在","多","多麽","多少","大","大家","她","她們","好","如","如上","如上所述","如下","如何","如其","如同","如是","如果","如此","如若","始而","孰料","孰知","寧","寧可","寧願","寧肯","它","它們","對","對於","對待","對方","對比","將","小","爾","爾後","爾爾","尚且","就","就是","就是了","就是說","就算","就要","盡","盡管","盡管如此","豈但","己","已","已矣","巴","巴巴","年","並","並且","庶乎","庶幾","開外","開始","歸","歸齊","當","當地","當然","當著","彼","彼時","彼此","往","待","很","得","得了","怎","怎麽","怎麽辦","怎麽樣","怎奈","怎樣","總之","總的來看","總的來說","總的說來","總而言之","恰恰相反","您","惟其","慢說","我","我們","或","或則","或是","或曰","或者","截至","所","所以","所在","所幸","所有","才","才能","打","打從","把","抑或","拿","按","按照","換句話說","換言之","據","據此","接著","故","故此","故而","旁人","無","無寧","無論","既","既往","既是","既然","日","時","時候","是","是以","是的","更","曾","替","替代","最","月","有","有些","有關","有及","有時","有的","望","朝","朝著","本","本人","本地","本著","本身","來","來著","來自","來說","極了","果然","果真","某","某個","某些","某某","根據","歟","正值","正如","正巧","正是","此","此地","此處","此外","此時","此次","此間","毋寧","每","每當","比","比及","比如","比方","沒奈何","沿","沿著","漫說","焉","然則","然後","然而","照","照著","猶且","猶自","甚且","甚麽","甚或","甚而","甚至","甚至於","用","用來","由","由於","由是","由此","由此可見","的","的確","的話","直到","相對而言","省得","看","眨眼","著","著呢","矣","矣乎","矣哉","離","秒","竟而","第","等","等到","等等","簡言之","管","類如","緊接著","縱","縱令","縱使","縱然","經","經過","結果","給","繼之","繼後","繼而","綜上所述","罷了","者","而","而且","而況","而後","而外","而已","而是","而言","能","能否","騰","自","自個兒","自從","自各兒","自後","自家","自己","自打","自身","至","至於","至今","至若","致","般的","若","若夫","若是","若果","若非","莫不然","莫如","莫若","雖","雖則","雖然","雖說","被","要","要不","要不是","要不然","要麽","要是","譬喻","譬如","讓","許多","論","設使","設或","設若","誠如","誠然","該","說","說來","請","諸","諸位","諸如","誰","誰人","誰料","誰知","賊死","賴以","趕","起","起見","趁","趁著","越是","距","跟","較","較之","邊","過","還","還是","還有","還要","這","這一來","這個","這麽","這麽些","這麽樣","這麽點兒","這些","這會兒","這兒","這就是說","這時","這樣","這次","這般","這邊","這里","進而","連","連同","逐步","通過","遵循","遵照","那","那個","那麽","那麽些","那麽樣","那些","那會兒","那兒","那時","那樣","那般","那邊","那里","都","鄙人","鑒於","針對","阿","除","除了","除外","除開","除此之外","除非","隨","隨後","隨時","隨著","難道說","零","非","非但","非徒","非特","非獨","靠","順","順著","首先"," ","︿","！","＃","＄","％","＆","（","）","＊","＋","，","０","１","２","３","４","５","６","７","８","９","：","；","＜","＞","？","＠","［","］","｛","｜","｝","～","￥"]

# the vectorizer object will be used to transform text to vector form
vectorizer = TfidfVectorizer(tokenizer=jieba.lcut, token_pattern='\w+|\$[\d\.]+|\S+', stop_words = [" "])

# apply transformation
tf_5 = vectorizer.fit_transform(texts)

# tf_feature_names tells us what word each column in the matric represents
tf_feature_names5 = vectorizer.get_feature_names()

#Distribute topics
lda_tfidf = LatentDirichletAllocation(n_components=2, random_state=0)
lda_tfidf.fit(tf_5)

#Display result
vis = pyLDAvis.sklearn.prepare(lda_tfidf, tf_5, vectorizer)
pyLDAvis.enable_notebook()
pyLDAvis.display(vis)

In [ ]:
plot_top_words(lda_tfidf,tf_feature_names5,30)

[8462 2794  250 2331 1510 1899 4939 1419 4662 2329 6374 3728 2215 3851
 2118 3831 8094 6339 2965 6186 6293 3350 3517 6663 8045  527 3835  977
 4738 1922]
['香港', '山竹', '中國', '天氣', '印尼', '回顧', '海嘯', '十大', '死亡', '天文台', '聖誕', '搜尋', '大事', '政府', '增至', '改革開放', '除夕', '習近平', '年度', '綠色', '美國', '應對', '投資', '菲律賓', '防災', '今年', '攻略', '全球', '氣候變化', '國家']
[26.16350118 15.93219474 15.71192651 14.80553179 13.5922339  13.38647755
 12.46470025 12.29179706 10.89120018 10.68089521 10.15828423 10.10409692
 10.0310155   9.86769705  9.84936839  9.5722952   9.52388233  9.38129367
  9.10540175  8.91291438  8.80975938  8.56707131  8.51698975  8.28683973
  8.14802489  8.09461029  8.07364506  8.06091575  8.04323432  7.73563762]
[5631 4939 1510 5128  250 2965 1874 6164 4072  854 2789 2004 6523 4665
  577 2794 5181 1942 2815 3588 3075  977 2118 5590 5509 5596 4051 1175
 5651 6186]
['發展', '海嘯', '印尼', '漢字', '中國', '年度', '四川', '經濟', '日本', '傾瀉', '山泥', '地震', '至少', '死傷', '企業', '山竹', '火山', '國際', '峰會', '持續', '引發', '全球', '增至',

## Anomaly6

In [ ]:
texts = anomaly6['keywords'].astype(str).to_list()
#chinese_stopwords=["、","。","〈","〉","《","》","一","一些","一何","一切","一則","一方面","一旦","一來","一樣","一般","一轉眼","七","萬一","三","上","上下","下","不","不僅","不但","不光","不單","不只","不外乎","不如","不妨","不盡","不盡然","不得","不怕","不惟","不成","不拘","不料","不是","不比","不然","不特","不獨","不管","不至於","不若","不論","不過","不問","與","與其","與其說","與否","與此同時","且","且不說","且說","兩者","個","個別","中","臨","為","為了","為什麽","為何","為止","為此","為著","乃","乃至","乃至於","麽","之","之一","之所以","之類","烏乎","乎","乘","九","也","也好","也罷","了","二","二來","於","於是","於是乎","雲雲","雲爾","五","些","亦","人","人們","人家","什","什麽","什麽樣","今","介於","仍","仍舊","從","從此","從而","他","他人","他們","他們們","以","以上","以為","以便","以免","以及","以故","以期","以來","以至","以至於","以致","們","任","任何","任憑","會","似的","但","但凡","但是","何","何以","何況","何處","何時","余外","作為","你","你們","使","使得","例如","依","依據","依照","便於","俺","俺們","倘","倘使","倘或","倘然","倘若","借","借儻然","假使","假如","假若","做","像","兒","先不先","光是","全體","全部","八","六","兮","共","關於","關於具體地說","其","其一","其中","其二","其他","其余","其它","其次","具體地說","具體說來","兼之","內","再","再其次","再則","再有","再者","再者說","再說","冒","沖","況且","幾","幾時","凡","凡是","憑","憑借","出於","出來","分","分別","則","則甚","別","別人","別處","別是","別的","別管","別說","到","前後","前此","前者","加之","加以","即","即令","即使","即便","即如","即或","即若","卻","去","又","又及","及","及其","及至","反之","反而","反過來","反過來說","受到","另","另一方面","另外","另悉","只","只當","只怕","只是","只有","只消","只要","只限","叫","叮咚","可","可以","可是","可見","各","各個","各位","各種","各自","同","同時","後","後者","向","向使","向著","嚇","嗎","否則","吧","吧噠","含","吱","呀","呃","嘔","唄","嗚","嗚呼","呢","呵","呵呵","呸","呼哧","咋","和","咚","咦","咧","咱","咱們","咳","哇","哈","哈哈","哉","哎","哎呀","哎喲","嘩","喲","哦","哩","哪","哪個","哪些","哪兒","哪天","哪年","哪怕","哪樣","哪邊","哪里","哼","哼唷","唉","唯有","啊","啐","啥","啦","啪達","啷當","喂","喏","喔唷","嘍","嗡","嗡嗡","嗬","嗯","噯","嘎","嘎登","噓","嘛","嘻","嘿","嘿嘿","四","因","因為","因了","因此","因著","因而","固然","在","在下","在於","地","基於","處在","多","多麽","多少","大","大家","她","她們","好","如","如上","如上所述","如下","如何","如其","如同","如是","如果","如此","如若","始而","孰料","孰知","寧","寧可","寧願","寧肯","它","它們","對","對於","對待","對方","對比","將","小","爾","爾後","爾爾","尚且","就","就是","就是了","就是說","就算","就要","盡","盡管","盡管如此","豈但","己","已","已矣","巴","巴巴","年","並","並且","庶乎","庶幾","開外","開始","歸","歸齊","當","當地","當然","當著","彼","彼時","彼此","往","待","很","得","得了","怎","怎麽","怎麽辦","怎麽樣","怎奈","怎樣","總之","總的來看","總的來說","總的說來","總而言之","恰恰相反","您","惟其","慢說","我","我們","或","或則","或是","或曰","或者","截至","所","所以","所在","所幸","所有","才","才能","打","打從","把","抑或","拿","按","按照","換句話說","換言之","據","據此","接著","故","故此","故而","旁人","無","無寧","無論","既","既往","既是","既然","日","時","時候","是","是以","是的","更","曾","替","替代","最","月","有","有些","有關","有及","有時","有的","望","朝","朝著","本","本人","本地","本著","本身","來","來著","來自","來說","極了","果然","果真","某","某個","某些","某某","根據","歟","正值","正如","正巧","正是","此","此地","此處","此外","此時","此次","此間","毋寧","每","每當","比","比及","比如","比方","沒奈何","沿","沿著","漫說","焉","然則","然後","然而","照","照著","猶且","猶自","甚且","甚麽","甚或","甚而","甚至","甚至於","用","用來","由","由於","由是","由此","由此可見","的","的確","的話","直到","相對而言","省得","看","眨眼","著","著呢","矣","矣乎","矣哉","離","秒","竟而","第","等","等到","等等","簡言之","管","類如","緊接著","縱","縱令","縱使","縱然","經","經過","結果","給","繼之","繼後","繼而","綜上所述","罷了","者","而","而且","而況","而後","而外","而已","而是","而言","能","能否","騰","自","自個兒","自從","自各兒","自後","自家","自己","自打","自身","至","至於","至今","至若","致","般的","若","若夫","若是","若果","若非","莫不然","莫如","莫若","雖","雖則","雖然","雖說","被","要","要不","要不是","要不然","要麽","要是","譬喻","譬如","讓","許多","論","設使","設或","設若","誠如","誠然","該","說","說來","請","諸","諸位","諸如","誰","誰人","誰料","誰知","賊死","賴以","趕","起","起見","趁","趁著","越是","距","跟","較","較之","邊","過","還","還是","還有","還要","這","這一來","這個","這麽","這麽些","這麽樣","這麽點兒","這些","這會兒","這兒","這就是說","這時","這樣","這次","這般","這邊","這里","進而","連","連同","逐步","通過","遵循","遵照","那","那個","那麽","那麽些","那麽樣","那些","那會兒","那兒","那時","那樣","那般","那邊","那里","都","鄙人","鑒於","針對","阿","除","除了","除外","除開","除此之外","除非","隨","隨後","隨時","隨著","難道說","零","非","非但","非徒","非特","非獨","靠","順","順著","首先"," ","︿","！","＃","＄","％","＆","（","）","＊","＋","，","０","１","２","３","４","５","６","７","８","９","：","；","＜","＞","？","＠","［","］","｛","｜","｝","～","￥"]

# the vectorizer object will be used to transform text to vector form
vectorizer = TfidfVectorizer(tokenizer=jieba.lcut, token_pattern='\w+|\$[\d\.]+|\S+', stop_words = [" "])

# apply transformation
tf_6 = vectorizer.fit_transform(texts)

# tf_feature_names tells us what word each column in the matric represents
tf_feature_names6 = vectorizer.get_feature_names()

#Distribute topics
lda_tfidf = LatentDirichletAllocation(n_components=2, random_state=0)
lda_tfidf.fit(tf_6)

#Display result
vis = pyLDAvis.sklearn.prepare(lda_tfidf, tf_6, vectorizer)
pyLDAvis.enable_notebook()
pyLDAvis.display(vis)

In [ ]:
plot_top_words(lda_tfidf,tf_feature_names6,30)

[ 5873 11500  3264  3256  6942 11928  9937  5944  7509  3328 11506  6797
  4342  6585  3098  3793  2223  4268   217  6481   752  4220  6050  7698
  3946  3267  4996 11658  6688  2974]
['暴雨', '雷暴', '天氣', '天文台', '深圳', '驟雨', '警告', '最高', '狂風', '失蹤', '雷雨', '洪水', '影響', '氣溫', '多雲', '小時', '取消', '引發', '下午', '死亡', '今年', '廣東', '未來', '生效', '工人', '天氣預報', '持續', '預警', '沖走', '增至']
[80.14730256 69.19841187 64.66454106 60.50300652 55.38318192 46.55523157
 41.31013584 39.55962752 35.19916911 35.03906302 31.75496529 29.66662545
 25.61764013 23.28611946 22.48946187 22.31720267 21.87365709 21.40332862
 21.36404942 21.1913542  20.60469945 20.35374226 20.03894952 19.61908213
 19.33341744 18.70838894 18.68532794 18.29370358 17.48382603 17.14043636]
[   44   101   374  7879  9891  2355  2733  8900  1354 11857  8698  1181
 11953  5415  4996  4883   336  8672  3908 11639   752  8558  3912  2714
  8011 10050  8424  7665  8853  1865]
['一帶', '一路', '中國', '發展', '論壇', '合作', '國際', '習近平', '全球', '香港', '綠色', '傾瀉', '高峰', '政

## Anomaly7

In [ ]:
texts = anomaly7['keywords'].astype(str).to_list()
#chinese_stopwords=["、","。","〈","〉","《","》","一","一些","一何","一切","一則","一方面","一旦","一來","一樣","一般","一轉眼","七","萬一","三","上","上下","下","不","不僅","不但","不光","不單","不只","不外乎","不如","不妨","不盡","不盡然","不得","不怕","不惟","不成","不拘","不料","不是","不比","不然","不特","不獨","不管","不至於","不若","不論","不過","不問","與","與其","與其說","與否","與此同時","且","且不說","且說","兩者","個","個別","中","臨","為","為了","為什麽","為何","為止","為此","為著","乃","乃至","乃至於","麽","之","之一","之所以","之類","烏乎","乎","乘","九","也","也好","也罷","了","二","二來","於","於是","於是乎","雲雲","雲爾","五","些","亦","人","人們","人家","什","什麽","什麽樣","今","介於","仍","仍舊","從","從此","從而","他","他人","他們","他們們","以","以上","以為","以便","以免","以及","以故","以期","以來","以至","以至於","以致","們","任","任何","任憑","會","似的","但","但凡","但是","何","何以","何況","何處","何時","余外","作為","你","你們","使","使得","例如","依","依據","依照","便於","俺","俺們","倘","倘使","倘或","倘然","倘若","借","借儻然","假使","假如","假若","做","像","兒","先不先","光是","全體","全部","八","六","兮","共","關於","關於具體地說","其","其一","其中","其二","其他","其余","其它","其次","具體地說","具體說來","兼之","內","再","再其次","再則","再有","再者","再者說","再說","冒","沖","況且","幾","幾時","凡","凡是","憑","憑借","出於","出來","分","分別","則","則甚","別","別人","別處","別是","別的","別管","別說","到","前後","前此","前者","加之","加以","即","即令","即使","即便","即如","即或","即若","卻","去","又","又及","及","及其","及至","反之","反而","反過來","反過來說","受到","另","另一方面","另外","另悉","只","只當","只怕","只是","只有","只消","只要","只限","叫","叮咚","可","可以","可是","可見","各","各個","各位","各種","各自","同","同時","後","後者","向","向使","向著","嚇","嗎","否則","吧","吧噠","含","吱","呀","呃","嘔","唄","嗚","嗚呼","呢","呵","呵呵","呸","呼哧","咋","和","咚","咦","咧","咱","咱們","咳","哇","哈","哈哈","哉","哎","哎呀","哎喲","嘩","喲","哦","哩","哪","哪個","哪些","哪兒","哪天","哪年","哪怕","哪樣","哪邊","哪里","哼","哼唷","唉","唯有","啊","啐","啥","啦","啪達","啷當","喂","喏","喔唷","嘍","嗡","嗡嗡","嗬","嗯","噯","嘎","嘎登","噓","嘛","嘻","嘿","嘿嘿","四","因","因為","因了","因此","因著","因而","固然","在","在下","在於","地","基於","處在","多","多麽","多少","大","大家","她","她們","好","如","如上","如上所述","如下","如何","如其","如同","如是","如果","如此","如若","始而","孰料","孰知","寧","寧可","寧願","寧肯","它","它們","對","對於","對待","對方","對比","將","小","爾","爾後","爾爾","尚且","就","就是","就是了","就是說","就算","就要","盡","盡管","盡管如此","豈但","己","已","已矣","巴","巴巴","年","並","並且","庶乎","庶幾","開外","開始","歸","歸齊","當","當地","當然","當著","彼","彼時","彼此","往","待","很","得","得了","怎","怎麽","怎麽辦","怎麽樣","怎奈","怎樣","總之","總的來看","總的來說","總的說來","總而言之","恰恰相反","您","惟其","慢說","我","我們","或","或則","或是","或曰","或者","截至","所","所以","所在","所幸","所有","才","才能","打","打從","把","抑或","拿","按","按照","換句話說","換言之","據","據此","接著","故","故此","故而","旁人","無","無寧","無論","既","既往","既是","既然","日","時","時候","是","是以","是的","更","曾","替","替代","最","月","有","有些","有關","有及","有時","有的","望","朝","朝著","本","本人","本地","本著","本身","來","來著","來自","來說","極了","果然","果真","某","某個","某些","某某","根據","歟","正值","正如","正巧","正是","此","此地","此處","此外","此時","此次","此間","毋寧","每","每當","比","比及","比如","比方","沒奈何","沿","沿著","漫說","焉","然則","然後","然而","照","照著","猶且","猶自","甚且","甚麽","甚或","甚而","甚至","甚至於","用","用來","由","由於","由是","由此","由此可見","的","的確","的話","直到","相對而言","省得","看","眨眼","著","著呢","矣","矣乎","矣哉","離","秒","竟而","第","等","等到","等等","簡言之","管","類如","緊接著","縱","縱令","縱使","縱然","經","經過","結果","給","繼之","繼後","繼而","綜上所述","罷了","者","而","而且","而況","而後","而外","而已","而是","而言","能","能否","騰","自","自個兒","自從","自各兒","自後","自家","自己","自打","自身","至","至於","至今","至若","致","般的","若","若夫","若是","若果","若非","莫不然","莫如","莫若","雖","雖則","雖然","雖說","被","要","要不","要不是","要不然","要麽","要是","譬喻","譬如","讓","許多","論","設使","設或","設若","誠如","誠然","該","說","說來","請","諸","諸位","諸如","誰","誰人","誰料","誰知","賊死","賴以","趕","起","起見","趁","趁著","越是","距","跟","較","較之","邊","過","還","還是","還有","還要","這","這一來","這個","這麽","這麽些","這麽樣","這麽點兒","這些","這會兒","這兒","這就是說","這時","這樣","這次","這般","這邊","這里","進而","連","連同","逐步","通過","遵循","遵照","那","那個","那麽","那麽些","那麽樣","那些","那會兒","那兒","那時","那樣","那般","那邊","那里","都","鄙人","鑒於","針對","阿","除","除了","除外","除開","除此之外","除非","隨","隨後","隨時","隨著","難道說","零","非","非但","非徒","非特","非獨","靠","順","順著","首先"," ","︿","！","＃","＄","％","＆","（","）","＊","＋","，","０","１","２","３","４","５","６","７","８","９","：","；","＜","＞","？","＠","［","］","｛","｜","｝","～","￥"]

# the vectorizer object will be used to transform text to vector form
vectorizer = TfidfVectorizer(tokenizer=jieba.lcut, token_pattern='\w+|\$[\d\.]+|\S+', stop_words = [" "])

# apply transformation
tf_7 = vectorizer.fit_transform(texts)

# tf_feature_names tells us what word each column in the matric represents
tf_feature_names7 = vectorizer.get_feature_names()

#Distribute topics
lda_tfidf = LatentDirichletAllocation(n_components=2, random_state=0)
lda_tfidf.fit(tf_7)

#Display result
vis = pyLDAvis.sklearn.prepare(lda_tfidf, tf_7, vectorizer)
pyLDAvis.enable_notebook()
pyLDAvis.display(vis)

In [ ]:
plot_top_words(lda_tfidf,tf_feature_names7,30)

[ 3010  3016 11325 10882 11133 10391  7132  5528  9484  7329 11265 11371
   751  2884  6131 10990 11123  6974  3912  5644  1012 11360  7499  4628
   853 10758  6793  3610  4967   755]
['天文台', '天氣', '驟雨', '雷暴', '風球', '酷熱', '狂風', '最高', '警告', '生效', '香港', '高溫', '今日', '多雲', '氣溫', '韋帕', '風暴', '熱帶', '幾陣', '未來', '信號', '高度', '發展', '持續', '低壓', '陽光', '澳門', '局部地區', '改發', '今晚']
[81.08519802 78.62173906 65.88753956 62.51523084 51.24063036 50.12059143
 41.45734927 39.56309969 39.52550759 36.70317792 34.09420719 31.25184303
 29.83696526 29.68123032 29.54941549 28.54509464 27.77997059 27.39388054
 27.02390937 24.49619645 24.33638384 24.00789788 23.60669081 23.10737434
 22.7881612  22.61440445 22.41667241 22.1482223  21.93279047 20.11575493]
[ 5456 11147   386  5275  6022  1150   529  3083   471  3671  4628  9616
  7484  6399  1993  2164  7489   747  8733  2556 11058  6403  7446  8915
   104  6216  2090  2756  5457  5181]
['暴雨', '颱風', '中國', '日本', '死亡', '傾瀉', '九州', '失蹤', '丹娜', '山泥', '持續', '貴州', '登陸', '洪水

## Anomaly8

In [ ]:
texts = anomaly8['keywords'].astype(str).to_list()
#chinese_stopwords=["、","。","〈","〉","《","》","一","一些","一何","一切","一則","一方面","一旦","一來","一樣","一般","一轉眼","七","萬一","三","上","上下","下","不","不僅","不但","不光","不單","不只","不外乎","不如","不妨","不盡","不盡然","不得","不怕","不惟","不成","不拘","不料","不是","不比","不然","不特","不獨","不管","不至於","不若","不論","不過","不問","與","與其","與其說","與否","與此同時","且","且不說","且說","兩者","個","個別","中","臨","為","為了","為什麽","為何","為止","為此","為著","乃","乃至","乃至於","麽","之","之一","之所以","之類","烏乎","乎","乘","九","也","也好","也罷","了","二","二來","於","於是","於是乎","雲雲","雲爾","五","些","亦","人","人們","人家","什","什麽","什麽樣","今","介於","仍","仍舊","從","從此","從而","他","他人","他們","他們們","以","以上","以為","以便","以免","以及","以故","以期","以來","以至","以至於","以致","們","任","任何","任憑","會","似的","但","但凡","但是","何","何以","何況","何處","何時","余外","作為","你","你們","使","使得","例如","依","依據","依照","便於","俺","俺們","倘","倘使","倘或","倘然","倘若","借","借儻然","假使","假如","假若","做","像","兒","先不先","光是","全體","全部","八","六","兮","共","關於","關於具體地說","其","其一","其中","其二","其他","其余","其它","其次","具體地說","具體說來","兼之","內","再","再其次","再則","再有","再者","再者說","再說","冒","沖","況且","幾","幾時","凡","凡是","憑","憑借","出於","出來","分","分別","則","則甚","別","別人","別處","別是","別的","別管","別說","到","前後","前此","前者","加之","加以","即","即令","即使","即便","即如","即或","即若","卻","去","又","又及","及","及其","及至","反之","反而","反過來","反過來說","受到","另","另一方面","另外","另悉","只","只當","只怕","只是","只有","只消","只要","只限","叫","叮咚","可","可以","可是","可見","各","各個","各位","各種","各自","同","同時","後","後者","向","向使","向著","嚇","嗎","否則","吧","吧噠","含","吱","呀","呃","嘔","唄","嗚","嗚呼","呢","呵","呵呵","呸","呼哧","咋","和","咚","咦","咧","咱","咱們","咳","哇","哈","哈哈","哉","哎","哎呀","哎喲","嘩","喲","哦","哩","哪","哪個","哪些","哪兒","哪天","哪年","哪怕","哪樣","哪邊","哪里","哼","哼唷","唉","唯有","啊","啐","啥","啦","啪達","啷當","喂","喏","喔唷","嘍","嗡","嗡嗡","嗬","嗯","噯","嘎","嘎登","噓","嘛","嘻","嘿","嘿嘿","四","因","因為","因了","因此","因著","因而","固然","在","在下","在於","地","基於","處在","多","多麽","多少","大","大家","她","她們","好","如","如上","如上所述","如下","如何","如其","如同","如是","如果","如此","如若","始而","孰料","孰知","寧","寧可","寧願","寧肯","它","它們","對","對於","對待","對方","對比","將","小","爾","爾後","爾爾","尚且","就","就是","就是了","就是說","就算","就要","盡","盡管","盡管如此","豈但","己","已","已矣","巴","巴巴","年","並","並且","庶乎","庶幾","開外","開始","歸","歸齊","當","當地","當然","當著","彼","彼時","彼此","往","待","很","得","得了","怎","怎麽","怎麽辦","怎麽樣","怎奈","怎樣","總之","總的來看","總的來說","總的說來","總而言之","恰恰相反","您","惟其","慢說","我","我們","或","或則","或是","或曰","或者","截至","所","所以","所在","所幸","所有","才","才能","打","打從","把","抑或","拿","按","按照","換句話說","換言之","據","據此","接著","故","故此","故而","旁人","無","無寧","無論","既","既往","既是","既然","日","時","時候","是","是以","是的","更","曾","替","替代","最","月","有","有些","有關","有及","有時","有的","望","朝","朝著","本","本人","本地","本著","本身","來","來著","來自","來說","極了","果然","果真","某","某個","某些","某某","根據","歟","正值","正如","正巧","正是","此","此地","此處","此外","此時","此次","此間","毋寧","每","每當","比","比及","比如","比方","沒奈何","沿","沿著","漫說","焉","然則","然後","然而","照","照著","猶且","猶自","甚且","甚麽","甚或","甚而","甚至","甚至於","用","用來","由","由於","由是","由此","由此可見","的","的確","的話","直到","相對而言","省得","看","眨眼","著","著呢","矣","矣乎","矣哉","離","秒","竟而","第","等","等到","等等","簡言之","管","類如","緊接著","縱","縱令","縱使","縱然","經","經過","結果","給","繼之","繼後","繼而","綜上所述","罷了","者","而","而且","而況","而後","而外","而已","而是","而言","能","能否","騰","自","自個兒","自從","自各兒","自後","自家","自己","自打","自身","至","至於","至今","至若","致","般的","若","若夫","若是","若果","若非","莫不然","莫如","莫若","雖","雖則","雖然","雖說","被","要","要不","要不是","要不然","要麽","要是","譬喻","譬如","讓","許多","論","設使","設或","設若","誠如","誠然","該","說","說來","請","諸","諸位","諸如","誰","誰人","誰料","誰知","賊死","賴以","趕","起","起見","趁","趁著","越是","距","跟","較","較之","邊","過","還","還是","還有","還要","這","這一來","這個","這麽","這麽些","這麽樣","這麽點兒","這些","這會兒","這兒","這就是說","這時","這樣","這次","這般","這邊","這里","進而","連","連同","逐步","通過","遵循","遵照","那","那個","那麽","那麽些","那麽樣","那些","那會兒","那兒","那時","那樣","那般","那邊","那里","都","鄙人","鑒於","針對","阿","除","除了","除外","除開","除此之外","除非","隨","隨後","隨時","隨著","難道說","零","非","非但","非徒","非特","非獨","靠","順","順著","首先"," ","︿","！","＃","＄","％","＆","（","）","＊","＋","，","０","１","２","３","４","５","６","７","８","９","：","；","＜","＞","？","＠","［","］","｛","｜","｝","～","￥"]

# the vectorizer object will be used to transform text to vector form
vectorizer = TfidfVectorizer(tokenizer=jieba.lcut, token_pattern='\w+|\$[\d\.]+|\S+', stop_words = [" "])

# apply transformation
tf_8 = vectorizer.fit_transform(texts)

# tf_feature_names tells us what word each column in the matric represents
tf_feature_names8 = vectorizer.get_feature_names()

#Distribute topics
lda_tfidf = LatentDirichletAllocation(n_components=2, random_state=0)
lda_tfidf.fit(tf_8)

#Display result
vis = pyLDAvis.sklearn.prepare(lda_tfidf, tf_8, vectorizer)
pyLDAvis.enable_notebook()
pyLDAvis.display(vis)

In [ ]:
plot_top_words(lda_tfidf,tf_feature_names8,30)

[ 242 1685 3426 4772 5575 7092 1277 5285 2407 6237 6614  798 5924 2991
  220 3353 1258  415 4650 5108 2396 3322 3696 3650 4256 1261 1951 3681
 1779 4821]
['中國', '國際', '日本', '發展', '舉行', '香港', '即位', '綠色', '巡遊', '路障', '金融', '全球', '解放軍', '持續', '世界', '新聞', '博會', '亮相', '環保', '第二屆', '峰會', '新世界', '東京', '未來', '清理', '博覽會', '大灣', '李克強', '基金', '皇德仁']
[19.99700114 15.46635722 14.86275929 13.95727344 13.79984252 13.58969697
 12.61303871 11.60960773 11.08670637 10.54175131 10.22782664 10.01259199
  9.1530986   8.93475159  8.7794381   8.70973799  8.63842259  8.53602654
  8.52214091  8.22636029  7.87527106  7.36476435  7.35475793  7.27939246
  7.25872653  7.09571926  7.06346343  6.8890057   6.79453215  6.78373445]
[2084 4772 2991 7092  492 1401 3994  242 3991 2772 1615  798 1685 2928
 3522 6392 5307 1993 5555  348 1996 3523 1038 6004 1954 3968 5460  397
 5263 1672]
['威尼斯', '發展', '持續', '香港', '企業', '合作', '水災', '中國', '水浸', '意大利', '嚴重', '全球', '國際', '投資', '暴雨', '進入', '緊急狀態', '天晴', '至少', '乾燥', '天氣', '暴雨成災', 

## Anomaly9

In [ ]:
texts = anomaly9['keywords'].astype(str).to_list()
#chinese_stopwords=["、","。","〈","〉","《","》","一","一些","一何","一切","一則","一方面","一旦","一來","一樣","一般","一轉眼","七","萬一","三","上","上下","下","不","不僅","不但","不光","不單","不只","不外乎","不如","不妨","不盡","不盡然","不得","不怕","不惟","不成","不拘","不料","不是","不比","不然","不特","不獨","不管","不至於","不若","不論","不過","不問","與","與其","與其說","與否","與此同時","且","且不說","且說","兩者","個","個別","中","臨","為","為了","為什麽","為何","為止","為此","為著","乃","乃至","乃至於","麽","之","之一","之所以","之類","烏乎","乎","乘","九","也","也好","也罷","了","二","二來","於","於是","於是乎","雲雲","雲爾","五","些","亦","人","人們","人家","什","什麽","什麽樣","今","介於","仍","仍舊","從","從此","從而","他","他人","他們","他們們","以","以上","以為","以便","以免","以及","以故","以期","以來","以至","以至於","以致","們","任","任何","任憑","會","似的","但","但凡","但是","何","何以","何況","何處","何時","余外","作為","你","你們","使","使得","例如","依","依據","依照","便於","俺","俺們","倘","倘使","倘或","倘然","倘若","借","借儻然","假使","假如","假若","做","像","兒","先不先","光是","全體","全部","八","六","兮","共","關於","關於具體地說","其","其一","其中","其二","其他","其余","其它","其次","具體地說","具體說來","兼之","內","再","再其次","再則","再有","再者","再者說","再說","冒","沖","況且","幾","幾時","凡","凡是","憑","憑借","出於","出來","分","分別","則","則甚","別","別人","別處","別是","別的","別管","別說","到","前後","前此","前者","加之","加以","即","即令","即使","即便","即如","即或","即若","卻","去","又","又及","及","及其","及至","反之","反而","反過來","反過來說","受到","另","另一方面","另外","另悉","只","只當","只怕","只是","只有","只消","只要","只限","叫","叮咚","可","可以","可是","可見","各","各個","各位","各種","各自","同","同時","後","後者","向","向使","向著","嚇","嗎","否則","吧","吧噠","含","吱","呀","呃","嘔","唄","嗚","嗚呼","呢","呵","呵呵","呸","呼哧","咋","和","咚","咦","咧","咱","咱們","咳","哇","哈","哈哈","哉","哎","哎呀","哎喲","嘩","喲","哦","哩","哪","哪個","哪些","哪兒","哪天","哪年","哪怕","哪樣","哪邊","哪里","哼","哼唷","唉","唯有","啊","啐","啥","啦","啪達","啷當","喂","喏","喔唷","嘍","嗡","嗡嗡","嗬","嗯","噯","嘎","嘎登","噓","嘛","嘻","嘿","嘿嘿","四","因","因為","因了","因此","因著","因而","固然","在","在下","在於","地","基於","處在","多","多麽","多少","大","大家","她","她們","好","如","如上","如上所述","如下","如何","如其","如同","如是","如果","如此","如若","始而","孰料","孰知","寧","寧可","寧願","寧肯","它","它們","對","對於","對待","對方","對比","將","小","爾","爾後","爾爾","尚且","就","就是","就是了","就是說","就算","就要","盡","盡管","盡管如此","豈但","己","已","已矣","巴","巴巴","年","並","並且","庶乎","庶幾","開外","開始","歸","歸齊","當","當地","當然","當著","彼","彼時","彼此","往","待","很","得","得了","怎","怎麽","怎麽辦","怎麽樣","怎奈","怎樣","總之","總的來看","總的來說","總的說來","總而言之","恰恰相反","您","惟其","慢說","我","我們","或","或則","或是","或曰","或者","截至","所","所以","所在","所幸","所有","才","才能","打","打從","把","抑或","拿","按","按照","換句話說","換言之","據","據此","接著","故","故此","故而","旁人","無","無寧","無論","既","既往","既是","既然","日","時","時候","是","是以","是的","更","曾","替","替代","最","月","有","有些","有關","有及","有時","有的","望","朝","朝著","本","本人","本地","本著","本身","來","來著","來自","來說","極了","果然","果真","某","某個","某些","某某","根據","歟","正值","正如","正巧","正是","此","此地","此處","此外","此時","此次","此間","毋寧","每","每當","比","比及","比如","比方","沒奈何","沿","沿著","漫說","焉","然則","然後","然而","照","照著","猶且","猶自","甚且","甚麽","甚或","甚而","甚至","甚至於","用","用來","由","由於","由是","由此","由此可見","的","的確","的話","直到","相對而言","省得","看","眨眼","著","著呢","矣","矣乎","矣哉","離","秒","竟而","第","等","等到","等等","簡言之","管","類如","緊接著","縱","縱令","縱使","縱然","經","經過","結果","給","繼之","繼後","繼而","綜上所述","罷了","者","而","而且","而況","而後","而外","而已","而是","而言","能","能否","騰","自","自個兒","自從","自各兒","自後","自家","自己","自打","自身","至","至於","至今","至若","致","般的","若","若夫","若是","若果","若非","莫不然","莫如","莫若","雖","雖則","雖然","雖說","被","要","要不","要不是","要不然","要麽","要是","譬喻","譬如","讓","許多","論","設使","設或","設若","誠如","誠然","該","說","說來","請","諸","諸位","諸如","誰","誰人","誰料","誰知","賊死","賴以","趕","起","起見","趁","趁著","越是","距","跟","較","較之","邊","過","還","還是","還有","還要","這","這一來","這個","這麽","這麽些","這麽樣","這麽點兒","這些","這會兒","這兒","這就是說","這時","這樣","這次","這般","這邊","這里","進而","連","連同","逐步","通過","遵循","遵照","那","那個","那麽","那麽些","那麽樣","那些","那會兒","那兒","那時","那樣","那般","那邊","那里","都","鄙人","鑒於","針對","阿","除","除了","除外","除開","除此之外","除非","隨","隨後","隨時","隨著","難道說","零","非","非但","非徒","非特","非獨","靠","順","順著","首先"," ","︿","！","＃","＄","％","＆","（","）","＊","＋","，","０","１","２","３","４","５","６","７","８","９","：","；","＜","＞","？","＠","［","］","｛","｜","｝","～","￥"]

# the vectorizer object will be used to transform text to vector form
vectorizer = TfidfVectorizer(tokenizer=jieba.lcut, token_pattern='\w+|\$[\d\.]+|\S+', stop_words = [" "])

# apply transformation
tf_9 = vectorizer.fit_transform(texts)

# tf_feature_names tells us what word each column in the matric represents
tf_feature_names9 = vectorizer.get_feature_names()

#Distribute topics
lda_tfidf = LatentDirichletAllocation(n_components=2, random_state=0)
lda_tfidf.fit(tf_9)

#Display result
vis = pyLDAvis.sklearn.prepare(lda_tfidf, tf_9, vectorizer)
pyLDAvis.enable_notebook()
pyLDAvis.display(vis)

In [ ]:
plot_top_words(lda_tfidf,tf_feature_names9,30)

[4193 3054 1500 2352 3099 3652 4091 3623 4017 2608 5437 1434 5327 2677
 2817 4296  653 5516 3172 5492  625 5385 2918 3061 5068 2146 3780  391
  751 3482]
['肺炎', '武漢', '天氣', '持續', '氣溫', '發展', '罷工', '疫情', '經濟', '新冠', '風暴', '多雲', '雷暴', '日本', '最新', '英國', '全球', '驟雨', '法治', '香港', '內地', '項目', '東京', '死亡', '醫管局', '情人節', '確診', '企業', '冷鋒', '狂風']
[22.7117871  14.7962142  13.95923402 12.11395863 11.4467156  11.42776138
 11.0751255  11.00017054 10.68698703 10.47466373 10.29933445  9.96868688
  9.62183015  9.56760938  9.48042799  9.15872728  9.05108089  8.87426293
  8.71986076  8.51098414  8.40396877  8.2141112   8.18591121  8.1555543
  7.75654424  7.7231661   7.69543951  7.58740205  7.42240479  7.41505955]
[5395 1819 4680 3343 3623 4630 4028 2782  179 5492 1497 1349 2540 3097
 1828 2630 1032 2031 4463 2629  555 3271 5437 3568 3652  376 1679  309
 4549 1256]
['預算案', '山火', '財政', '澳洲', '疫情', '警告', '綠色', '暴雨', '中國', '香港', '天文台', '基金', '政府', '氣候變化', '岑智明', '新聞', '取消', '影響', '裁員', '新省', '億元', '減薪', '風暴', 

## Anomaly10

In [ ]:
texts = anomaly10['keywords'].astype(str).to_list()
#chinese_stopwords=["、","。","〈","〉","《","》","一","一些","一何","一切","一則","一方面","一旦","一來","一樣","一般","一轉眼","七","萬一","三","上","上下","下","不","不僅","不但","不光","不單","不只","不外乎","不如","不妨","不盡","不盡然","不得","不怕","不惟","不成","不拘","不料","不是","不比","不然","不特","不獨","不管","不至於","不若","不論","不過","不問","與","與其","與其說","與否","與此同時","且","且不說","且說","兩者","個","個別","中","臨","為","為了","為什麽","為何","為止","為此","為著","乃","乃至","乃至於","麽","之","之一","之所以","之類","烏乎","乎","乘","九","也","也好","也罷","了","二","二來","於","於是","於是乎","雲雲","雲爾","五","些","亦","人","人們","人家","什","什麽","什麽樣","今","介於","仍","仍舊","從","從此","從而","他","他人","他們","他們們","以","以上","以為","以便","以免","以及","以故","以期","以來","以至","以至於","以致","們","任","任何","任憑","會","似的","但","但凡","但是","何","何以","何況","何處","何時","余外","作為","你","你們","使","使得","例如","依","依據","依照","便於","俺","俺們","倘","倘使","倘或","倘然","倘若","借","借儻然","假使","假如","假若","做","像","兒","先不先","光是","全體","全部","八","六","兮","共","關於","關於具體地說","其","其一","其中","其二","其他","其余","其它","其次","具體地說","具體說來","兼之","內","再","再其次","再則","再有","再者","再者說","再說","冒","沖","況且","幾","幾時","凡","凡是","憑","憑借","出於","出來","分","分別","則","則甚","別","別人","別處","別是","別的","別管","別說","到","前後","前此","前者","加之","加以","即","即令","即使","即便","即如","即或","即若","卻","去","又","又及","及","及其","及至","反之","反而","反過來","反過來說","受到","另","另一方面","另外","另悉","只","只當","只怕","只是","只有","只消","只要","只限","叫","叮咚","可","可以","可是","可見","各","各個","各位","各種","各自","同","同時","後","後者","向","向使","向著","嚇","嗎","否則","吧","吧噠","含","吱","呀","呃","嘔","唄","嗚","嗚呼","呢","呵","呵呵","呸","呼哧","咋","和","咚","咦","咧","咱","咱們","咳","哇","哈","哈哈","哉","哎","哎呀","哎喲","嘩","喲","哦","哩","哪","哪個","哪些","哪兒","哪天","哪年","哪怕","哪樣","哪邊","哪里","哼","哼唷","唉","唯有","啊","啐","啥","啦","啪達","啷當","喂","喏","喔唷","嘍","嗡","嗡嗡","嗬","嗯","噯","嘎","嘎登","噓","嘛","嘻","嘿","嘿嘿","四","因","因為","因了","因此","因著","因而","固然","在","在下","在於","地","基於","處在","多","多麽","多少","大","大家","她","她們","好","如","如上","如上所述","如下","如何","如其","如同","如是","如果","如此","如若","始而","孰料","孰知","寧","寧可","寧願","寧肯","它","它們","對","對於","對待","對方","對比","將","小","爾","爾後","爾爾","尚且","就","就是","就是了","就是說","就算","就要","盡","盡管","盡管如此","豈但","己","已","已矣","巴","巴巴","年","並","並且","庶乎","庶幾","開外","開始","歸","歸齊","當","當地","當然","當著","彼","彼時","彼此","往","待","很","得","得了","怎","怎麽","怎麽辦","怎麽樣","怎奈","怎樣","總之","總的來看","總的來說","總的說來","總而言之","恰恰相反","您","惟其","慢說","我","我們","或","或則","或是","或曰","或者","截至","所","所以","所在","所幸","所有","才","才能","打","打從","把","抑或","拿","按","按照","換句話說","換言之","據","據此","接著","故","故此","故而","旁人","無","無寧","無論","既","既往","既是","既然","日","時","時候","是","是以","是的","更","曾","替","替代","最","月","有","有些","有關","有及","有時","有的","望","朝","朝著","本","本人","本地","本著","本身","來","來著","來自","來說","極了","果然","果真","某","某個","某些","某某","根據","歟","正值","正如","正巧","正是","此","此地","此處","此外","此時","此次","此間","毋寧","每","每當","比","比及","比如","比方","沒奈何","沿","沿著","漫說","焉","然則","然後","然而","照","照著","猶且","猶自","甚且","甚麽","甚或","甚而","甚至","甚至於","用","用來","由","由於","由是","由此","由此可見","的","的確","的話","直到","相對而言","省得","看","眨眼","著","著呢","矣","矣乎","矣哉","離","秒","竟而","第","等","等到","等等","簡言之","管","類如","緊接著","縱","縱令","縱使","縱然","經","經過","結果","給","繼之","繼後","繼而","綜上所述","罷了","者","而","而且","而況","而後","而外","而已","而是","而言","能","能否","騰","自","自個兒","自從","自各兒","自後","自家","自己","自打","自身","至","至於","至今","至若","致","般的","若","若夫","若是","若果","若非","莫不然","莫如","莫若","雖","雖則","雖然","雖說","被","要","要不","要不是","要不然","要麽","要是","譬喻","譬如","讓","許多","論","設使","設或","設若","誠如","誠然","該","說","說來","請","諸","諸位","諸如","誰","誰人","誰料","誰知","賊死","賴以","趕","起","起見","趁","趁著","越是","距","跟","較","較之","邊","過","還","還是","還有","還要","這","這一來","這個","這麽","這麽些","這麽樣","這麽點兒","這些","這會兒","這兒","這就是說","這時","這樣","這次","這般","這邊","這里","進而","連","連同","逐步","通過","遵循","遵照","那","那個","那麽","那麽些","那麽樣","那些","那會兒","那兒","那時","那樣","那般","那邊","那里","都","鄙人","鑒於","針對","阿","除","除了","除外","除開","除此之外","除非","隨","隨後","隨時","隨著","難道說","零","非","非但","非徒","非特","非獨","靠","順","順著","首先"," ","︿","！","＃","＄","％","＆","（","）","＊","＋","，","０","１","２","３","４","５","６","７","８","９","：","；","＜","＞","？","＠","［","］","｛","｜","｝","～","￥"]

# the vectorizer object will be used to transform text to vector form
vectorizer = TfidfVectorizer(tokenizer=jieba.lcut, token_pattern='\w+|\$[\d\.]+|\S+', stop_words = [" "])

# apply transformation
tf_10 = vectorizer.fit_transform(texts)

# tf_feature_names tells us what word each column in the matric represents
tf_feature_names10 = vectorizer.get_feature_names()

#Distribute topics
lda_tfidf = LatentDirichletAllocation(n_components=2, random_state=0)
lda_tfidf.fit(tf_10)

#Display result
vis = pyLDAvis.sklearn.prepare(lda_tfidf, tf_10, vectorizer)
pyLDAvis.enable_notebook()
pyLDAvis.display(vis)

In [ ]:
plot_top_words(lda_tfidf,tf_feature_names10,30)

[ 2771 10115 10101   880   164  5750  5879  4569  3678  2777  6398  7766
  1264  6680  5651  7569    95  5262  4916  4780 10091  6840  5960   655
  1295  4315  5475  9239  5896 10212]
['天文台', '颱風', '風球', '信號', '三號', '沙德爾', '浪卡', '改發', '強風', '天氣', '熱帶', '考慮', '八號', '生效', '氣旋', '維持', '一號', '本港', '明日', '新聞', '風暴', '發出', '消息', '今日', '公里', '掛號', '機會', '逼近', '浪卡襲', '香港']
[116.02867242  75.27801599  75.15278519  59.3213883   51.8561299
  51.58829575  49.26837666  41.84094541  39.59446237  38.77571877
  38.63166269  36.91289835  35.7565145   35.55301688  34.6362126
  32.81276488  30.94523613  30.05853537  29.31787304  28.84127743
  28.3884404   26.77304337  26.58629385  25.4314269   25.39744584
  24.91795651  24.22357103  24.13429361  24.12318085  23.55347466]
[ 6844   337 10212  4231  6003   495  2011  2772  7743  2727  7559  1191
  9519  2339  5654  6805  2628  5206  6644  4140  7528  5129   719 10289
  2471  1551  8439  5098  9528  1272]
['發展', '中國', '香港', '持續', '深圳', '乾燥', '合作', '天晴', '習近平

## Anomaly11

In [ ]:
texts = anomaly11['keywords'].astype(str).to_list()
#chinese_stopwords=["、","。","〈","〉","《","》","一","一些","一何","一切","一則","一方面","一旦","一來","一樣","一般","一轉眼","七","萬一","三","上","上下","下","不","不僅","不但","不光","不單","不只","不外乎","不如","不妨","不盡","不盡然","不得","不怕","不惟","不成","不拘","不料","不是","不比","不然","不特","不獨","不管","不至於","不若","不論","不過","不問","與","與其","與其說","與否","與此同時","且","且不說","且說","兩者","個","個別","中","臨","為","為了","為什麽","為何","為止","為此","為著","乃","乃至","乃至於","麽","之","之一","之所以","之類","烏乎","乎","乘","九","也","也好","也罷","了","二","二來","於","於是","於是乎","雲雲","雲爾","五","些","亦","人","人們","人家","什","什麽","什麽樣","今","介於","仍","仍舊","從","從此","從而","他","他人","他們","他們們","以","以上","以為","以便","以免","以及","以故","以期","以來","以至","以至於","以致","們","任","任何","任憑","會","似的","但","但凡","但是","何","何以","何況","何處","何時","余外","作為","你","你們","使","使得","例如","依","依據","依照","便於","俺","俺們","倘","倘使","倘或","倘然","倘若","借","借儻然","假使","假如","假若","做","像","兒","先不先","光是","全體","全部","八","六","兮","共","關於","關於具體地說","其","其一","其中","其二","其他","其余","其它","其次","具體地說","具體說來","兼之","內","再","再其次","再則","再有","再者","再者說","再說","冒","沖","況且","幾","幾時","凡","凡是","憑","憑借","出於","出來","分","分別","則","則甚","別","別人","別處","別是","別的","別管","別說","到","前後","前此","前者","加之","加以","即","即令","即使","即便","即如","即或","即若","卻","去","又","又及","及","及其","及至","反之","反而","反過來","反過來說","受到","另","另一方面","另外","另悉","只","只當","只怕","只是","只有","只消","只要","只限","叫","叮咚","可","可以","可是","可見","各","各個","各位","各種","各自","同","同時","後","後者","向","向使","向著","嚇","嗎","否則","吧","吧噠","含","吱","呀","呃","嘔","唄","嗚","嗚呼","呢","呵","呵呵","呸","呼哧","咋","和","咚","咦","咧","咱","咱們","咳","哇","哈","哈哈","哉","哎","哎呀","哎喲","嘩","喲","哦","哩","哪","哪個","哪些","哪兒","哪天","哪年","哪怕","哪樣","哪邊","哪里","哼","哼唷","唉","唯有","啊","啐","啥","啦","啪達","啷當","喂","喏","喔唷","嘍","嗡","嗡嗡","嗬","嗯","噯","嘎","嘎登","噓","嘛","嘻","嘿","嘿嘿","四","因","因為","因了","因此","因著","因而","固然","在","在下","在於","地","基於","處在","多","多麽","多少","大","大家","她","她們","好","如","如上","如上所述","如下","如何","如其","如同","如是","如果","如此","如若","始而","孰料","孰知","寧","寧可","寧願","寧肯","它","它們","對","對於","對待","對方","對比","將","小","爾","爾後","爾爾","尚且","就","就是","就是了","就是說","就算","就要","盡","盡管","盡管如此","豈但","己","已","已矣","巴","巴巴","年","並","並且","庶乎","庶幾","開外","開始","歸","歸齊","當","當地","當然","當著","彼","彼時","彼此","往","待","很","得","得了","怎","怎麽","怎麽辦","怎麽樣","怎奈","怎樣","總之","總的來看","總的來說","總的說來","總而言之","恰恰相反","您","惟其","慢說","我","我們","或","或則","或是","或曰","或者","截至","所","所以","所在","所幸","所有","才","才能","打","打從","把","抑或","拿","按","按照","換句話說","換言之","據","據此","接著","故","故此","故而","旁人","無","無寧","無論","既","既往","既是","既然","日","時","時候","是","是以","是的","更","曾","替","替代","最","月","有","有些","有關","有及","有時","有的","望","朝","朝著","本","本人","本地","本著","本身","來","來著","來自","來說","極了","果然","果真","某","某個","某些","某某","根據","歟","正值","正如","正巧","正是","此","此地","此處","此外","此時","此次","此間","毋寧","每","每當","比","比及","比如","比方","沒奈何","沿","沿著","漫說","焉","然則","然後","然而","照","照著","猶且","猶自","甚且","甚麽","甚或","甚而","甚至","甚至於","用","用來","由","由於","由是","由此","由此可見","的","的確","的話","直到","相對而言","省得","看","眨眼","著","著呢","矣","矣乎","矣哉","離","秒","竟而","第","等","等到","等等","簡言之","管","類如","緊接著","縱","縱令","縱使","縱然","經","經過","結果","給","繼之","繼後","繼而","綜上所述","罷了","者","而","而且","而況","而後","而外","而已","而是","而言","能","能否","騰","自","自個兒","自從","自各兒","自後","自家","自己","自打","自身","至","至於","至今","至若","致","般的","若","若夫","若是","若果","若非","莫不然","莫如","莫若","雖","雖則","雖然","雖說","被","要","要不","要不是","要不然","要麽","要是","譬喻","譬如","讓","許多","論","設使","設或","設若","誠如","誠然","該","說","說來","請","諸","諸位","諸如","誰","誰人","誰料","誰知","賊死","賴以","趕","起","起見","趁","趁著","越是","距","跟","較","較之","邊","過","還","還是","還有","還要","這","這一來","這個","這麽","這麽些","這麽樣","這麽點兒","這些","這會兒","這兒","這就是說","這時","這樣","這次","這般","這邊","這里","進而","連","連同","逐步","通過","遵循","遵照","那","那個","那麽","那麽些","那麽樣","那些","那會兒","那兒","那時","那樣","那般","那邊","那里","都","鄙人","鑒於","針對","阿","除","除了","除外","除開","除此之外","除非","隨","隨後","隨時","隨著","難道說","零","非","非但","非徒","非特","非獨","靠","順","順著","首先"," ","︿","！","＃","＄","％","＆","（","）","＊","＋","，","０","１","２","３","４","５","６","７","８","９","：","；","＜","＞","？","＠","［","］","｛","｜","｝","～","￥"]

# the vectorizer object will be used to transform text to vector form
vectorizer = TfidfVectorizer(tokenizer=jieba.lcut, token_pattern='\w+|\$[\d\.]+|\S+', stop_words = [" "])

# apply transformation
tf_11 = vectorizer.fit_transform(texts)

# tf_feature_names tells us what word each column in the matric represents
tf_feature_names11 = vectorizer.get_feature_names()

#Distribute topics
lda_tfidf = LatentDirichletAllocation(n_components=2, random_state=0)
lda_tfidf.fit(tf_11)

#Display result
vis = pyLDAvis.sklearn.prepare(lda_tfidf, tf_11, vectorizer)
pyLDAvis.enable_notebook()
pyLDAvis.display(vis)

In [ ]:
plot_top_words(lda_tfidf,tf_feature_names11,30)

[ 2799  8466  5012   889  6602  2804  6745 10052 10229  5628  9746  2785
  6283  4764  3688 10242 10243  9721   649    84  5565  1922  2401  3253
  5196  5175 10013  1013   957  5587]
['天文台', '警告', '暴雨', '信號', '生效', '天氣', '發出', '香港', '黃色', '水浸', '雷暴', '大雨', '熱帶', '新聞', '影響', '黑色', '黑雨', '雨量', '今年', '一號', '毫米', '取消', '地區', '小時', '本港', '未來', '首個', '傾瀉', '停課', '氣旋']
[93.04562865 72.90236973 70.57353155 34.86067862 33.77397823 31.5481846
 29.03850943 26.10115796 25.35613051 23.90881811 23.8995282  23.30714026
 22.67480646 22.54769947 22.42485693 22.34676132 22.30998993 22.22541467
 22.09181568 20.92009677 20.91891708 20.90533795 19.92679985 19.92135594
 18.87782613 17.97653048 16.49850911 15.76361435 15.55697982 15.49121246]
[10104  5075  4222  2804  6752  9746  7455  5590   339  2649  9259   651
  4148 10052  9347  3309  4887  1163  9616  2474 10130  2029  8329  2763
  2360  3554  6558  7433  6955  2490]
['驟雨', '最高', '持續', '天氣', '發展', '雷暴', '綠色', '氣溫', '中國', '多雲', '酷熱', '今日', '投資', '香港', 

## Anomaly12

In [ ]:
texts = anomaly12['keywords'].astype(str).to_list()
#chinese_stopwords=["、","。","〈","〉","《","》","一","一些","一何","一切","一則","一方面","一旦","一來","一樣","一般","一轉眼","七","萬一","三","上","上下","下","不","不僅","不但","不光","不單","不只","不外乎","不如","不妨","不盡","不盡然","不得","不怕","不惟","不成","不拘","不料","不是","不比","不然","不特","不獨","不管","不至於","不若","不論","不過","不問","與","與其","與其說","與否","與此同時","且","且不說","且說","兩者","個","個別","中","臨","為","為了","為什麽","為何","為止","為此","為著","乃","乃至","乃至於","麽","之","之一","之所以","之類","烏乎","乎","乘","九","也","也好","也罷","了","二","二來","於","於是","於是乎","雲雲","雲爾","五","些","亦","人","人們","人家","什","什麽","什麽樣","今","介於","仍","仍舊","從","從此","從而","他","他人","他們","他們們","以","以上","以為","以便","以免","以及","以故","以期","以來","以至","以至於","以致","們","任","任何","任憑","會","似的","但","但凡","但是","何","何以","何況","何處","何時","余外","作為","你","你們","使","使得","例如","依","依據","依照","便於","俺","俺們","倘","倘使","倘或","倘然","倘若","借","借儻然","假使","假如","假若","做","像","兒","先不先","光是","全體","全部","八","六","兮","共","關於","關於具體地說","其","其一","其中","其二","其他","其余","其它","其次","具體地說","具體說來","兼之","內","再","再其次","再則","再有","再者","再者說","再說","冒","沖","況且","幾","幾時","凡","凡是","憑","憑借","出於","出來","分","分別","則","則甚","別","別人","別處","別是","別的","別管","別說","到","前後","前此","前者","加之","加以","即","即令","即使","即便","即如","即或","即若","卻","去","又","又及","及","及其","及至","反之","反而","反過來","反過來說","受到","另","另一方面","另外","另悉","只","只當","只怕","只是","只有","只消","只要","只限","叫","叮咚","可","可以","可是","可見","各","各個","各位","各種","各自","同","同時","後","後者","向","向使","向著","嚇","嗎","否則","吧","吧噠","含","吱","呀","呃","嘔","唄","嗚","嗚呼","呢","呵","呵呵","呸","呼哧","咋","和","咚","咦","咧","咱","咱們","咳","哇","哈","哈哈","哉","哎","哎呀","哎喲","嘩","喲","哦","哩","哪","哪個","哪些","哪兒","哪天","哪年","哪怕","哪樣","哪邊","哪里","哼","哼唷","唉","唯有","啊","啐","啥","啦","啪達","啷當","喂","喏","喔唷","嘍","嗡","嗡嗡","嗬","嗯","噯","嘎","嘎登","噓","嘛","嘻","嘿","嘿嘿","四","因","因為","因了","因此","因著","因而","固然","在","在下","在於","地","基於","處在","多","多麽","多少","大","大家","她","她們","好","如","如上","如上所述","如下","如何","如其","如同","如是","如果","如此","如若","始而","孰料","孰知","寧","寧可","寧願","寧肯","它","它們","對","對於","對待","對方","對比","將","小","爾","爾後","爾爾","尚且","就","就是","就是了","就是說","就算","就要","盡","盡管","盡管如此","豈但","己","已","已矣","巴","巴巴","年","並","並且","庶乎","庶幾","開外","開始","歸","歸齊","當","當地","當然","當著","彼","彼時","彼此","往","待","很","得","得了","怎","怎麽","怎麽辦","怎麽樣","怎奈","怎樣","總之","總的來看","總的來說","總的說來","總而言之","恰恰相反","您","惟其","慢說","我","我們","或","或則","或是","或曰","或者","截至","所","所以","所在","所幸","所有","才","才能","打","打從","把","抑或","拿","按","按照","換句話說","換言之","據","據此","接著","故","故此","故而","旁人","無","無寧","無論","既","既往","既是","既然","日","時","時候","是","是以","是的","更","曾","替","替代","最","月","有","有些","有關","有及","有時","有的","望","朝","朝著","本","本人","本地","本著","本身","來","來著","來自","來說","極了","果然","果真","某","某個","某些","某某","根據","歟","正值","正如","正巧","正是","此","此地","此處","此外","此時","此次","此間","毋寧","每","每當","比","比及","比如","比方","沒奈何","沿","沿著","漫說","焉","然則","然後","然而","照","照著","猶且","猶自","甚且","甚麽","甚或","甚而","甚至","甚至於","用","用來","由","由於","由是","由此","由此可見","的","的確","的話","直到","相對而言","省得","看","眨眼","著","著呢","矣","矣乎","矣哉","離","秒","竟而","第","等","等到","等等","簡言之","管","類如","緊接著","縱","縱令","縱使","縱然","經","經過","結果","給","繼之","繼後","繼而","綜上所述","罷了","者","而","而且","而況","而後","而外","而已","而是","而言","能","能否","騰","自","自個兒","自從","自各兒","自後","自家","自己","自打","自身","至","至於","至今","至若","致","般的","若","若夫","若是","若果","若非","莫不然","莫如","莫若","雖","雖則","雖然","雖說","被","要","要不","要不是","要不然","要麽","要是","譬喻","譬如","讓","許多","論","設使","設或","設若","誠如","誠然","該","說","說來","請","諸","諸位","諸如","誰","誰人","誰料","誰知","賊死","賴以","趕","起","起見","趁","趁著","越是","距","跟","較","較之","邊","過","還","還是","還有","還要","這","這一來","這個","這麽","這麽些","這麽樣","這麽點兒","這些","這會兒","這兒","這就是說","這時","這樣","這次","這般","這邊","這里","進而","連","連同","逐步","通過","遵循","遵照","那","那個","那麽","那麽些","那麽樣","那些","那會兒","那兒","那時","那樣","那般","那邊","那里","都","鄙人","鑒於","針對","阿","除","除了","除外","除開","除此之外","除非","隨","隨後","隨時","隨著","難道說","零","非","非但","非徒","非特","非獨","靠","順","順著","首先"," ","︿","！","＃","＄","％","＆","（","）","＊","＋","，","０","１","２","３","４","５","６","７","８","９","：","；","＜","＞","？","＠","［","］","｛","｜","｝","～","￥"]

# the vectorizer object will be used to transform text to vector form
vectorizer = TfidfVectorizer(tokenizer=jieba.lcut, token_pattern='\w+|\$[\d\.]+|\S+', stop_words = [" "])

# apply transformation
tf_12 = vectorizer.fit_transform(texts)

# tf_feature_names tells us what word each column in the matric represents
tf_feature_names12 = vectorizer.get_feature_names()

#Distribute topics
lda_tfidf = LatentDirichletAllocation(n_components=2, random_state=0)
lda_tfidf.fit(tf_12)

#Display result
vis = pyLDAvis.sklearn.prepare(lda_tfidf, tf_12, vectorizer)
pyLDAvis.enable_notebook()
pyLDAvis.display(vis)

In [ ]:
plot_top_words(lda_tfidf,tf_feature_names12,30)

[ 6627 13337  9014  3022 13706 13236  7291  3797  2215  7740  6713  7746
  2416  4918 10134  5263   434  5615 13197  1529  6628 13174  8972  2564
  9406 13341  4949 10477  7786  9342]
['暴雨', '颱風', '登陸', '四川', '黑格', '預警', '死亡', '失蹤', '北京', '洪水', '最高', '洪災', '南韓', '強降雨', '美國', '應急', '中國', '持續', '響應', '內地', '暴雨成災', '韓國', '疫情', '受災', '福建', '颶風', '影響', '至少', '浙江', '確診']
[81.34192864 65.67391808 51.07854106 41.1277383  37.8975487  37.72706202
 36.60172303 33.10025266 32.66198876 32.62356581 31.05246951 29.06524999
 27.27813714 26.64633713 25.65122017 25.56340609 25.25798228 25.09534105
 24.55538506 23.97049293 23.80379168 23.60394405 23.53248114 23.49574483
 22.66771037 21.85447114 21.83324997 21.30591322 21.09383705 19.29984676]
[ 3724 13585 13322  1184  3730 13337 13540 12411 13479 13064   200  1667
  8814  6031  6298 11335  8432   839  6871  4919   434 11051  8252  3384
  8047  9026   271  6837  7433 13579]
['天文台', '高斯', '風球', '信號', '天氣', '颱風', '驟雨', '酷熱', '香港', '雷暴', '三號', '八號', '生效', '改